<a href="https://colab.research.google.com/github/hg402/EPQ_Neg/blob/main/openai_api_sentiment_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install packages & input the API key

In [2]:
!pip install ipython-autotime
%load_ext autotime
!pip install openai
!pip install docx2txt==0.8

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.5/361.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3959 sha256=06946a7b22d607dae9c556840ddf896806152db5cbf899620c385522c0aa1464
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt
time: 20.6 s (started: 2024-08-16 03:38:41 +00:00)


In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass('Enter your OpenAI API KEY:')

Enter your OpenAI API KEY:··········
time: 25.8 s (started: 2024-08-10 16:09:50 +00:00)


# Load the transcripts

In [ ]:
import docx2txt
import os

text_list = []

# Directory to the folder with all transcripts in MS Word format (.docx)
directory = '/link/to/the/folder/containing/transcripts'

# sorted by file names
for filename in sorted(os.listdir(directory)):
  if filename.endswith(".docx"):
    filepath = os.path.join(directory, filename)
    text = docx2txt.process(filepath)
    text_list.append(text)

trimmed_text_list = []
for text in text_list:
  index = text.find("Transcript")
  if index != -1:
    trimmed_text = text[index:]
    trimmed_text_list.append(trimmed_text)

time: 33.6 s (started: 2024-08-10 15:53:56 +00:00)


## Client's overall tactic

In [ ]:
import csv
import json
from collections import Counter
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Function to get JSON response from OpenAI API
def get_predominant_tactic(text):
    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": (
                    "You have been assigned a task that is very important for your career. Please read the transcript provided below. The auditor is referred to as 'auditor' and the client as 'client 1/2/3/4' or 'speaker 1/2/3/4/…' in the transcript conversations."
                    "Use the following categorization schema to determine the predominant tactic of the client. Please also consider that there can be different descriptions for one tactic. "
                    "1. Select a position that is the most conservative. If necessary, you can concede to a less conservative position later in the negotiation. (Concessionary)"
                    "2. Try to get more information about the auditor’s preferences. (Integrative)"
                    "3. Threaten to terminate the relationship if the auditor does not agree with your solution. (Contending)"
                    "4. Whatever you communicate to the auditor, do not appear as if you will back down from that position. Appear firm. (Contending)"
                    "5. Attempt to trade off certain issues. There may be certain issues that are very important to you and not very important to the auditor and vice versa. (Integrative)"
                    "6. Provide substantive rationale for your solution to persuade the client to change their mind. (Contending)"
                    "7. Bring all concerns about the issue out into the open so that the issue could be resolved in the best possible way. (Integrative)"
                    "8. Try to find some middle ground to resolve the issue. (Compromise)"
                    "Contending, Concessionary, and Compromise are considered Distributive tactics. "
                    "Pay attention to only format your response in JSON format as below:"
                    "Set the Key as 'Predominant Tactic of the Client' and the Value as the tactic itself."
                )
            },
            {
                "role": "user",
                "content": text
            }
        ],
        temperature=0.7,
        max_tokens=64,
        top_p=1
    )

    json_object = json.loads(response.choices[0].message.content)
    return json_object["Predominant Tactic of the Client"]

# Function to determine majority tactic
def determine_majority_tactic(text):
    tactic_count = Counter()
    iterations = 0

    while True:
        tactic = get_predominant_tactic(text)
        tactic_count[tactic] += 1
        iterations += 1
        print(iterations)
        print(tactic)
        # Ensure at least two iterations before checking for majority
        if iterations > 1:
            most_common_tactic, count = tactic_count.most_common(1)[0]
            if count > iterations / 2:
                return most_common_tactic

# List to store extracted values
data = []

# Iterate over each item in trimmed_text_list and determine the majority tactic
for text in trimmed_text_list:
    majority_tactic = determine_majority_tactic(text)
    data.append([text, majority_tactic])

# Save data to CSV file
with open('/content/drive/MyDrive/EPQ & NEG/data/OpenAI API/majority_tactics.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Trimmed Text', 'Majority Tactic'])
    writer.writerows(data)

print("Data saved to majority_tactics.csv")


1
Contending
2
Integrative
3
Contending
1
Compromise
2
Compromise
1
Compromise
2
Compromise
1
Compromise
2
Compromise
1
Compromise
2
Compromise
1
Contending
2
Contending
1
Integrative
2
Compromise
3
Compromise
1
Integrative
2
Integrative
1
Integrative
2
Integrative
1
Integrative
2
Integrative
1
Compromise
2
Integrative
3
Compromise
1
Integrative
2
Compromise
3
Integrative
1
Integrative
2
Integrative
1
Integrative
2
Integrative
1
Compromise
2
Compromise
1
Integrative
2
Integrative
1
Compromise
2
Compromise
1
Integrative
2
Integrative
1
Compromise
2
Compromise
1
Compromise
2
Compromise
1
Compromise
2
Integrative
3
Integrative
1
Compromise
2
Compromise
Data saved to majority_tactics.csv
time: 6min 47s (started: 2024-07-09 02:29:37 +00:00)


## Professional skepticism questionnaire (three runs)

In [ ]:
import csv
import json
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Function to get auditor assessment scores from OpenAI API
def get_auditor_assessment_scores(text):
    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an assistant helping to evaluate the performance of an auditor. Read the provided audit negotiation transcript where the auditor is referred to as 'auditor' and the client as 'client 1/2/3/4' or 'speaker 1/2/3/4/…'."
                    "Background: Professional skepticism incorporates the attributes associated with being a skeptic to a professional setting that requires due diligence and a standard of care. Questioning mind is about maintaining an attitude of doubt and curiosity, and being prepared to challenge the status quo if the audit evidence suggests that there may be a problem. It’s about not taking things at face value, but digging deeper to ensure the financial statements present a true and fair view."
                    "Your task: Answer the following questions on a scale of 1 (strongly disagree), 2 (disagree), 3 (neither agree or disagree), 4 (agree), and 5 (strongly agree):"
                    "Q17. The auditor exercised professional skepticism while generating alternatives."
                    "Q18. While generating alternatives, the auditor’s attitude included a questioning mind."
                    "Q19. While generating alternatives, the auditor critically assessed the available audit evidence."
                    "Q20. While generating alternatives, the auditor was alert to conditions which may have indicated possible misstatement due to error or fraud."
                    "Provide your 1 to 5 integer responses in JSON format as below:"
                    "{\"Q17\": 0, \"Q18\": 0, \"Q19\": 0, \"Q20\": 0}"
                )
            },
            {
                "role": "user",
                "content": text
            }
        ],
        temperature=0.7,
        max_tokens=100,
        top_p=1
    )

    json_object = json.loads(response.choices[0].message.content)
    return json_object

# List to store extracted values
data = []

# Iterate over each item in trimmed_text_list and get assessment scores
for text in trimmed_text_list:
    assessment_scores = get_auditor_assessment_scores(text)
    data.append([assessment_scores["Q17"], assessment_scores["Q18"], assessment_scores["Q19"], assessment_scores["Q20"]])
    print(f"Q17: {assessment_scores['Q17']}, Q18: {assessment_scores['Q18']}, Q19: {assessment_scores['Q19']}, Q20: {assessment_scores['Q20']}")

# Save data to CSV file
with open('/content/drive/MyDrive/EPQ & NEG/data/OpenAI API/auditor_action_questionnaire_scores_0806v1.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Q17', 'Q18', 'Q19', 'Q20'])
    writer.writerows(data)

print("Data saved.")

Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 4, Q18: 4, Q19: 4, Q20: 3
Q17: 4, Q18: 4, Q19: 4, Q20: 3
Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 4, Q18: 4, Q19: 3, Q20: 3
Q17: 4, Q18: 4, Q19: 5, Q20: 4
Q17: 2, Q18: 2, Q19: 3, Q20: 2
Q17: 5, Q18: 5, Q19: 5, Q20: 5
Q17: 4, Q18: 4, Q19: 3, Q20: 3
Q17: 3, Q18: 3, Q19: 3, Q20: 2
Q17: 3, Q18: 3, Q19: 3, Q20: 3
Q17: 3, Q18: 3, Q19: 3, Q20: 3
Q17: 4, Q18: 4, Q19: 4, Q20: 3
Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 3, Q18: 3, Q19: 3, Q20: 4
Q17: 3, Q18: 3, Q19: 3, Q20: 3
Q17: 4, Q18: 4, Q19: 5, Q20: 5
Q17: 3, Q18: 3, Q19: 3, Q20: 3
Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 4, Q18: 4, Q19: 4, Q20: 4
Data saved.
time: 2min 27s (started: 2024-08-07 01:44:43 +00:00)


In [ ]:
import csv
import json
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Function to get auditor assessment scores from OpenAI API
def get_auditor_assessment_scores(text):
    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an assistant helping to evaluate the performance of an auditor. Read the provided audit negotiation transcript where the auditor is referred to as 'auditor' and the client as 'client 1/2/3/4' or 'speaker 1/2/3/4/…'."
                    "Background: Professional skepticism incorporates the attributes associated with being a skeptic to a professional setting that requires due diligence and a standard of care. Questioning mind is about maintaining an attitude of doubt and curiosity, and being prepared to challenge the status quo if the audit evidence suggests that there may be a problem. It’s about not taking things at face value, but digging deeper to ensure the financial statements present a true and fair view."
                    "Your task: Answer the following questions on a scale of 1 (strongly disagree), 2 (disagree), 3 (neither agree or disagree), 4 (agree), and 5 (strongly agree):"
                    "Q17. The auditor exercised professional skepticism while generating alternatives."
                    "Q18. While generating alternatives, the auditor’s attitude included a questioning mind."
                    "Q19. While generating alternatives, the auditor critically assessed the available audit evidence."
                    "Q20. While generating alternatives, the auditor was alert to conditions which may have indicated possible misstatement due to error or fraud."
                    "Provide your 1 to 5 integer responses in JSON format as below:"
                    "{\"Q17\": 0, \"Q18\": 0, \"Q19\": 0, \"Q20\": 0}"
                )
            },
            {
                "role": "user",
                "content": text
            }
        ],
        temperature=0.7,
        max_tokens=100,
        top_p=1
    )

    json_object = json.loads(response.choices[0].message.content)
    return json_object

# List to store extracted values
data = []

# Iterate over each item in trimmed_text_list and get assessment scores
for text in trimmed_text_list:
    assessment_scores = get_auditor_assessment_scores(text)
    data.append([assessment_scores["Q17"], assessment_scores["Q18"], assessment_scores["Q19"], assessment_scores["Q20"]])
    print(f"Q17: {assessment_scores['Q17']}, Q18: {assessment_scores['Q18']}, Q19: {assessment_scores['Q19']}, Q20: {assessment_scores['Q20']}")

# Save data to CSV file
with open('/content/drive/MyDrive/EPQ & NEG/data/OpenAI API/auditor_action_questionnaire_scores_0806v2.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Q17', 'Q18', 'Q19', 'Q20'])
    writer.writerows(data)

print("Data saved.")

Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 3, Q18: 4, Q19: 4, Q20: 3
Q17: 3, Q18: 3, Q19: 3, Q20: 3
Q17: 4, Q18: 5, Q19: 4, Q20: 4
Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 4, Q18: 4, Q19: 4, Q20: 3
Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 2, Q18: 3, Q19: 2, Q20: 2
Q17: 5, Q18: 5, Q19: 5, Q20: 5
Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 3, Q18: 3, Q19: 3, Q20: 3
Q17: 2, Q18: 2, Q19: 2, Q20: 2
Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 5, Q18: 5, Q19: 5, Q20: 5
Q17: 3, Q18: 4, Q19: 3, Q20: 3
Q17: 3, Q18: 3, Q19: 3, Q20: 3
Q17: 3, Q18: 3, Q19: 3, Q20: 3
Q17: 2, Q18: 2, Q19: 2, Q20: 2
Q17: 4, Q18: 4, Q19: 4, Q20: 3
Q17: 5, Q18: 5, Q19: 5, Q20: 5
Q17: 4, Q18: 4, Q19: 3, Q20: 4
Data saved.
time: 2min 48s (started: 2024-08-07 01:52:48 +00:00)


In [ ]:
import csv
import json
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Function to get auditor assessment scores from OpenAI API
def get_auditor_assessment_scores(text):
    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an assistant helping to evaluate the performance of an auditor. Read the provided audit negotiation transcript where the auditor is referred to as 'auditor' and the client as 'client 1/2/3/4' or 'speaker 1/2/3/4/…'."
                    "Background: Professional skepticism incorporates the attributes associated with being a skeptic to a professional setting that requires due diligence and a standard of care. Questioning mind is about maintaining an attitude of doubt and curiosity, and being prepared to challenge the status quo if the audit evidence suggests that there may be a problem. It’s about not taking things at face value, but digging deeper to ensure the financial statements present a true and fair view."
                    "Your task: Answer the following questions on a scale of 1 (strongly disagree), 2 (disagree), 3 (neither agree or disagree), 4 (agree), and 5 (strongly agree):"
                    "Q17. The auditor exercised professional skepticism while generating alternatives."
                    "Q18. While generating alternatives, the auditor’s attitude included a questioning mind."
                    "Q19. While generating alternatives, the auditor critically assessed the available audit evidence."
                    "Q20. While generating alternatives, the auditor was alert to conditions which may have indicated possible misstatement due to error or fraud."
                    "Provide your 1 to 5 integer responses in JSON format as below:"
                    "{\"Q17\": 0, \"Q18\": 0, \"Q19\": 0, \"Q20\": 0}"
                )
            },
            {
                "role": "user",
                "content": text
            }
        ],
        temperature=0.7,
        max_tokens=100,
        top_p=1
    )

    json_object = json.loads(response.choices[0].message.content)
    return json_object

# List to store extracted values
data = []

# Iterate over each item in trimmed_text_list and get assessment scores
for text in trimmed_text_list:
    assessment_scores = get_auditor_assessment_scores(text)
    data.append([assessment_scores["Q17"], assessment_scores["Q18"], assessment_scores["Q19"], assessment_scores["Q20"]])
    print(f"Q17: {assessment_scores['Q17']}, Q18: {assessment_scores['Q18']}, Q19: {assessment_scores['Q19']}, Q20: {assessment_scores['Q20']}")

# Save data to CSV file
with open('/content/drive/MyDrive/EPQ & NEG/data/OpenAI API/auditor_action_questionnaire_scores_0806v3.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Q17', 'Q18', 'Q19', 'Q20'])
    writer.writerows(data)

print("Data saved.")

Q17: 4, Q18: 5, Q19: 4, Q20: 4
Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 3, Q18: 3, Q19: 3, Q20: 3
Q17: 3, Q18: 4, Q19: 3, Q20: 2
Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 4, Q18: 4, Q19: 4, Q20: 3
Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 3, Q18: 3, Q19: 3, Q20: 3
Q17: 5, Q18: 5, Q19: 5, Q20: 5
Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 3, Q18: 3, Q19: 3, Q20: 3
Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 3, Q18: 3, Q19: 3, Q20: 3
Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 2, Q18: 2, Q19: 2, Q20: 2
Q17: 2, Q18: 2, Q19: 2, Q20: 2
Q17: 4, Q18: 4, Q19: 3, Q20: 3
Q17: 2, Q18: 3, Q19: 2, Q20: 2
Q17: 4, Q18: 4, Q19: 4, Q20: 3
Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 4, Q18: 4, Q19: 4, Q20: 4
Data saved.
time: 3min (started: 2024-08-07 01:55:36 +00:00)


### separate into Inventory and AR issues

In [ ]:
import csv
import json
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Function to get auditor assessment scores from OpenAI API
def get_auditor_assessment_scores(text):
    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an assistant helping to evaluate the performance of an auditor. Read the provided audit negotiation transcript where the auditor is referred to as 'auditor' and the client as 'client 1/2/3/4' or 'speaker 1/2/3/4/…'."
                    "Background: Professional skepticism incorporates the attributes associated with being a skeptic to a professional setting that requires due diligence and a standard of care. Questioning mind is about maintaining an attitude of doubt and curiosity, and being prepared to challenge the status quo if the audit evidence suggests that there may be a problem. It’s about not taking things at face value, but digging deeper to ensure the financial statements present a true and fair view."
                    "Your first task (do NOT print results): Separate the negotiation transcript into two parts: one for inventory write-off, the other for the reduction in allowance for doubtful accounts of accounts receivable (AR). The auditor is simply called “auditor” and clients are called “client 1/2/3/4” or “speaker 1/2/3/4/…” in the transcript conversations. "
                    "Your second task (print results in the designated format): For each part, answer the following questions on a scale of 1 (strongly disagree), 2 (disagree), 3 (neither agree or disagree), 4 (agree), and 5 (strongly agree):"
                    "Q17. The auditor exercised professional skepticism while generating alternatives."
                    "Q18. While generating alternatives, the auditor’s attitude included a questioning mind."
                    "Q19. While generating alternatives, the auditor critically assessed the available audit evidence."
                    "Q20. While generating alternatives, the auditor was alert to conditions which may have indicated possible misstatement due to error or fraud."
                    "Provide your 1 to 5 integer responses from the second task in JSON format as below:"
                    "{\"Q17_inventory\": 0, \"Q18_inventory\": 0, \"Q19_inventory\": 0, \"Q20_inventory\": 0, \"Q17_AR\": 0, \"Q18_AR\": 0, \"Q19_AR\": 0, \"Q20_AR\": 0, }"
                )
            },
            {
                "role": "user",
                "content": text
            }
        ],
        temperature=0.7,
        max_tokens=100,
        top_p=1
    )

    json_object = json.loads(response.choices[0].message.content)
    return json_object

# List to store extracted values
data = []

# Iterate over each item in trimmed_text_list and get assessment scores
for text in trimmed_text_list:
    assessment_scores = get_auditor_assessment_scores(text)
    data.append([assessment_scores["Q17_inventory"], assessment_scores["Q18_inventory"], assessment_scores["Q19_inventory"], assessment_scores["Q20_inventory"], assessment_scores["Q17_AR"], assessment_scores["Q18_AR"], assessment_scores["Q19_AR"], assessment_scores["Q20_AR"]])
    print(f"Q17: {assessment_scores['Q17_inventory']}, Q18: {assessment_scores['Q18_inventory']}, Q19: {assessment_scores['Q19_inventory']}, Q20: {assessment_scores['Q20_inventory']}, Q17: {assessment_scores['Q17_AR']}, Q18: {assessment_scores['Q18_AR']}, Q19: {assessment_scores['Q19_AR']}, Q20: {assessment_scores['Q20_AR']}")

# Save data to CSV file
with open('/content/drive/MyDrive/EPQ & NEG/data/OpenAI API/auditor_action_inv_ar_questionnaire_scores_0806v1.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Q17_inventory', 'Q18_inventory', 'Q19_inventory', 'Q20_inventory', 'Q17_AR', 'Q18_AR', 'Q19_AR', 'Q20_AR'])
    writer.writerows(data)

print("Data saved.")

Q17: 4, Q18: 5, Q19: 4, Q20: 4, Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 3, Q18: 3, Q19: 4, Q20: 4, Q17: 3, Q18: 3, Q19: 4, Q20: 4
Q17: 4, Q18: 4, Q19: 4, Q20: 4, Q17: 3, Q18: 3, Q19: 3, Q20: 3
Q17: 3, Q18: 3, Q19: 4, Q20: 4, Q17: 2, Q18: 3, Q19: 3, Q20: 3
Q17: 4, Q18: 4, Q19: 4, Q20: 4, Q17: 3, Q18: 3, Q19: 3, Q20: 3
Q17: 4, Q18: 4, Q19: 4, Q20: 4, Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 5, Q18: 5, Q19: 5, Q20: 5, Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 3, Q18: 3, Q19: 3, Q20: 3, Q17: 2, Q18: 2, Q19: 2, Q20: 2
Q17: 4, Q18: 4, Q19: 4, Q20: 4, Q17: 4, Q18: 4, Q19: 5, Q20: 4
Q17: 3, Q18: 3, Q19: 3, Q20: 3, Q17: 4, Q18: 4, Q19: 3, Q20: 4
Q17: 3, Q18: 3, Q19: 3, Q20: 3, Q17: 3, Q18: 3, Q19: 3, Q20: 3
Q17: 4, Q18: 4, Q19: 4, Q20: 4, Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 4, Q18: 4, Q19: 4, Q20: 5, Q17: 4, Q18: 4, Q19: 4, Q20: 5
Q17: 4, Q18: 4, Q19: 4, Q20: 4, Q17: 3, Q18: 3, Q19: 3, Q20: 3
Q17: 4, Q18: 5, Q19: 4, Q20: 4, Q17: 3, Q18: 4, Q19: 3, Q20: 3
Q17: 4, Q18: 4, Q19: 4, Q20: 4, Q17: 4, Q18: 4, Q19: 4,

In [ ]:
import csv
import json
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Function to get auditor assessment scores from OpenAI API
def get_auditor_assessment_scores(text):
    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an assistant helping to evaluate the performance of an auditor. Read the provided audit negotiation transcript where the auditor is referred to as 'auditor' and the client as 'client 1/2/3/4' or 'speaker 1/2/3/4/…'."
                    "Background: Professional skepticism incorporates the attributes associated with being a skeptic to a professional setting that requires due diligence and a standard of care. Questioning mind is about maintaining an attitude of doubt and curiosity, and being prepared to challenge the status quo if the audit evidence suggests that there may be a problem. It’s about not taking things at face value, but digging deeper to ensure the financial statements present a true and fair view."
                    "Your first task (do NOT print results): Separate the negotiation transcript into two parts: one for inventory write-off, the other for the reduction in allowance for doubtful accounts of accounts receivable (AR). The auditor is simply called “auditor” and clients are called “client 1/2/3/4” or “speaker 1/2/3/4/…” in the transcript conversations. "
                    "Your second task (print results in the designated format): For each part, answer the following questions on a scale of 1 (strongly disagree), 2 (disagree), 3 (neither agree or disagree), 4 (agree), and 5 (strongly agree):"
                    "Q17. The auditor exercised professional skepticism while generating alternatives."
                    "Q18. While generating alternatives, the auditor’s attitude included a questioning mind."
                    "Q19. While generating alternatives, the auditor critically assessed the available audit evidence."
                    "Q20. While generating alternatives, the auditor was alert to conditions which may have indicated possible misstatement due to error or fraud."
                    "Provide your 1 to 5 integer responses from the second task in JSON format as below:"
                    "{\"Q17_inventory\": 0, \"Q18_inventory\": 0, \"Q19_inventory\": 0, \"Q20_inventory\": 0, \"Q17_AR\": 0, \"Q18_AR\": 0, \"Q19_AR\": 0, \"Q20_AR\": 0, }"
                )
            },
            {
                "role": "user",
                "content": text
            }
        ],
        temperature=0.7,
        max_tokens=100,
        top_p=1
    )

    json_object = json.loads(response.choices[0].message.content)
    return json_object

# List to store extracted values
data = []

# Iterate over each item in trimmed_text_list and get assessment scores
for text in trimmed_text_list:
    assessment_scores = get_auditor_assessment_scores(text)
    data.append([assessment_scores["Q17_inventory"], assessment_scores["Q18_inventory"], assessment_scores["Q19_inventory"], assessment_scores["Q20_inventory"], assessment_scores["Q17_AR"], assessment_scores["Q18_AR"], assessment_scores["Q19_AR"], assessment_scores["Q20_AR"]])
    print(f"Q17: {assessment_scores['Q17_inventory']}, Q18: {assessment_scores['Q18_inventory']}, Q19: {assessment_scores['Q19_inventory']}, Q20: {assessment_scores['Q20_inventory']}, Q17: {assessment_scores['Q17_AR']}, Q18: {assessment_scores['Q18_AR']}, Q19: {assessment_scores['Q19_AR']}, Q20: {assessment_scores['Q20_AR']}")

# Save data to CSV file
with open('/content/drive/MyDrive/EPQ & NEG/data/OpenAI API/auditor_action_inv_ar_questionnaire_scores_0806v2.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Q17_inventory', 'Q18_inventory', 'Q19_inventory', 'Q20_inventory', 'Q17_AR', 'Q18_AR', 'Q19_AR', 'Q20_AR'])
    writer.writerows(data)

print("Data saved.")

Q17: 3, Q18: 3, Q19: 4, Q20: 4, Q17: 3, Q18: 4, Q19: 4, Q20: 3
Q17: 3, Q18: 3, Q19: 4, Q20: 4, Q17: 3, Q18: 3, Q19: 4, Q20: 4
Q17: 3, Q18: 3, Q19: 3, Q20: 3, Q17: 3, Q18: 3, Q19: 3, Q20: 3
Q17: 4, Q18: 4, Q19: 4, Q20: 4, Q17: 3, Q18: 3, Q19: 3, Q20: 3
Q17: 4, Q18: 4, Q19: 4, Q20: 4, Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 3, Q18: 4, Q19: 3, Q20: 3, Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 4, Q18: 4, Q19: 4, Q20: 4, Q17: 3, Q18: 3, Q19: 3, Q20: 3
Q17: 3, Q18: 3, Q19: 4, Q20: 4, Q17: 2, Q18: 2, Q19: 3, Q20: 2
Q17: 4, Q18: 4, Q19: 4, Q20: 4, Q17: 5, Q18: 5, Q19: 5, Q20: 5
Q17: 4, Q18: 4, Q19: 3, Q20: 4, Q17: 3, Q18: 3, Q19: 4, Q20: 4
Q17: 3, Q18: 3, Q19: 4, Q20: 4, Q17: 3, Q18: 3, Q19: 4, Q20: 4
Q17: 4, Q18: 4, Q19: 4, Q20: 4, Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 4, Q18: 4, Q19: 3, Q20: 3, Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 3, Q18: 3, Q19: 4, Q20: 4, Q17: 3, Q18: 3, Q19: 4, Q20: 4
Q17: 4, Q18: 3, Q19: 5, Q20: 4, Q17: 3, Q18: 3, Q19: 4, Q20: 4
Q17: 4, Q18: 4, Q19: 4, Q20: 5, Q17: 4, Q18: 4, Q19: 4,

In [ ]:
import csv
import json
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Function to get auditor assessment scores from OpenAI API
def get_auditor_assessment_scores(text):
    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an assistant helping to evaluate the performance of an auditor. Read the provided audit negotiation transcript where the auditor is referred to as 'auditor' and the client as 'client 1/2/3/4' or 'speaker 1/2/3/4/…'."
                    "Background: Professional skepticism incorporates the attributes associated with being a skeptic to a professional setting that requires due diligence and a standard of care. Questioning mind is about maintaining an attitude of doubt and curiosity, and being prepared to challenge the status quo if the audit evidence suggests that there may be a problem. It’s about not taking things at face value, but digging deeper to ensure the financial statements present a true and fair view."
                    "Your first task (do NOT print results): Separate the negotiation transcript into two parts: one for inventory write-off, the other for the reduction in allowance for doubtful accounts of accounts receivable (AR). The auditor is simply called “auditor” and clients are called “client 1/2/3/4” or “speaker 1/2/3/4/…” in the transcript conversations. "
                    "Your second task (print results in the designated format): For each part, answer the following questions on a scale of 1 (strongly disagree), 2 (disagree), 3 (neither agree or disagree), 4 (agree), and 5 (strongly agree):"
                    "Q17. The auditor exercised professional skepticism while generating alternatives."
                    "Q18. While generating alternatives, the auditor’s attitude included a questioning mind."
                    "Q19. While generating alternatives, the auditor critically assessed the available audit evidence."
                    "Q20. While generating alternatives, the auditor was alert to conditions which may have indicated possible misstatement due to error or fraud."
                    "Provide your 1 to 5 integer responses from the second task in JSON format as below:"
                    "{\"Q17_inventory\": 0, \"Q18_inventory\": 0, \"Q19_inventory\": 0, \"Q20_inventory\": 0, \"Q17_AR\": 0, \"Q18_AR\": 0, \"Q19_AR\": 0, \"Q20_AR\": 0, }"
                )
            },
            {
                "role": "user",
                "content": text
            }
        ],
        temperature=0.7,
        max_tokens=100,
        top_p=1
    )

    json_object = json.loads(response.choices[0].message.content)
    return json_object

# List to store extracted values
data = []

# Iterate over each item in trimmed_text_list and get assessment scores
for text in trimmed_text_list:
    assessment_scores = get_auditor_assessment_scores(text)
    data.append([assessment_scores["Q17_inventory"], assessment_scores["Q18_inventory"], assessment_scores["Q19_inventory"], assessment_scores["Q20_inventory"], assessment_scores["Q17_AR"], assessment_scores["Q18_AR"], assessment_scores["Q19_AR"], assessment_scores["Q20_AR"]])
    print(f"Q17: {assessment_scores['Q17_inventory']}, Q18: {assessment_scores['Q18_inventory']}, Q19: {assessment_scores['Q19_inventory']}, Q20: {assessment_scores['Q20_inventory']}, Q17: {assessment_scores['Q17_AR']}, Q18: {assessment_scores['Q18_AR']}, Q19: {assessment_scores['Q19_AR']}, Q20: {assessment_scores['Q20_AR']}")

# Save data to CSV file
with open('/content/drive/MyDrive/EPQ & NEG/data/OpenAI API/auditor_action_inv_ar_questionnaire_scores_0806v3.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Q17_inventory', 'Q18_inventory', 'Q19_inventory', 'Q20_inventory', 'Q17_AR', 'Q18_AR', 'Q19_AR', 'Q20_AR'])
    writer.writerows(data)

print("Data saved.")

Q17: 4, Q18: 4, Q19: 4, Q20: 4, Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 3, Q18: 4, Q19: 4, Q20: 3, Q17: 3, Q18: 4, Q19: 4, Q20: 3
Q17: 4, Q18: 4, Q19: 4, Q20: 4, Q17: 3, Q18: 3, Q19: 3, Q20: 3
Q17: 4, Q18: 4, Q19: 4, Q20: 4, Q17: 3, Q18: 3, Q19: 3, Q20: 3
Q17: 4, Q18: 4, Q19: 4, Q20: 4, Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 4, Q18: 4, Q19: 4, Q20: 4, Q17: 4, Q18: 4, Q19: 5, Q20: 5
Q17: 4, Q18: 4, Q19: 5, Q20: 5, Q17: 3, Q18: 3, Q19: 4, Q20: 4
Q17: 3, Q18: 3, Q19: 3, Q20: 3, Q17: 2, Q18: 2, Q19: 2, Q20: 2
Q17: 4, Q18: 4, Q19: 4, Q20: 4, Q17: 5, Q18: 5, Q19: 5, Q20: 5
Q17: 3, Q18: 4, Q19: 3, Q20: 3, Q17: 2, Q18: 3, Q19: 2, Q20: 2
Q17: 3, Q18: 3, Q19: 3, Q20: 3, Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 4, Q18: 4, Q19: 4, Q20: 4, Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 4, Q18: 4, Q19: 4, Q20: 4, Q17: 3, Q18: 3, Q19: 3, Q20: 3
Q17: 2, Q18: 2, Q19: 3, Q20: 3, Q17: 3, Q18: 3, Q19: 4, Q20: 4
Q17: 4, Q18: 4, Q19: 5, Q20: 5, Q17: 3, Q18: 4, Q19: 4, Q20: 4
Q17: 3, Q18: 3, Q19: 4, Q20: 4, Q17: 3, Q18: 3, Q19: 4,

## Ethicality

In [ ]:
import csv
import json
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Function to get auditor assessment scores from OpenAI API
def get_auditor_assessment_scores(text):
    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an assistant helping to evaluate the performance of an auditor. Read the provided audit negotiation transcript where the auditor is referred to as 'auditor' and the client as 'client 1/2/3/4' or 'speaker 1/2/3/4/…'."
                    "Background: There are two major dimensions underlying individual variations in moral judgments. Individuals high on relativism reject the possibility of formulating or relying on universal moral rules when drawing conclusions about moral questions. Individuals high on idealism assume that desirable consequences can, with the 'right' action, always be obtained. "
                    "Your task: Evaluate the following statements on a scale of Completely Disagree - 1, Largely Disagree - 2, Moderately Disagree - 3, Slightly Disagree - 4, Neither Agree nor Disagree - 5, Slightly Agree - 6, Moderately Agree - 7, Largely Agree - 8, and Completely Agree - 9:"
                    "Statement 1. The auditor demonstrates high relativism by evaluating moral issues in the context of specific situations rather than following consistent moral rules."
                    "Statement 2. The auditor demonstrates high idealism by believing that desirable outcomes can always be achieved through the 'right' actions"
                    "Provide your 1 to 9 integer responses in JSON format as below:"
                    "{\"Relativism\": 0, \"Idealism\": 0}"
                )
            },
            {
                "role": "user",
                "content": text
            }
        ],
        temperature=0.7,
        max_tokens=100,
        top_p=1
    )

    json_object = json.loads(response.choices[0].message.content)
    return json_object

# List to store extracted values
data = []

# Iterate over each item in trimmed_text_list and get assessment scores
for text in trimmed_text_list:
    assessment_scores = get_auditor_assessment_scores(text)
    data.append([assessment_scores["Relativism"], assessment_scores["Idealism"]])
    print(f"Relativism: {assessment_scores['Relativism']}, Idealism: {assessment_scores['Idealism']}")

# Save data to CSV file
with open('/content/drive/MyDrive/EPQ & NEG/data/OpenAI API/auditor_epq_scores_0801.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Relativism', 'Idealism'])
    writer.writerows(data)

print("Data saved to auditor_epq_scores_0801.csv")

Relativism: 8, Idealism: 7
Relativism: 8, Idealism: 3
Relativism: 6, Idealism: 4
Relativism: 8, Idealism: 4
Relativism: 4, Idealism: 5
Relativism: 7, Idealism: 5
Relativism: 5, Idealism: 3
Relativism: 6, Idealism: 5
Relativism: 6, Idealism: 5
Relativism: 4, Idealism: 5
Relativism: 6, Idealism: 4
Relativism: 6, Idealism: 3
Relativism: 8, Idealism: 3
Relativism: 6, Idealism: 3
Relativism: 6, Idealism: 4
Relativism: 4, Idealism: 5
Relativism: 5, Idealism: 3
Relativism: 7, Idealism: 4
Relativism: 5, Idealism: 4
Relativism: 4, Idealism: 6
Relativism: 8, Idealism: 4
Relativism: 6, Idealism: 3
Data saved to auditor_epq_scores_0801.csv
time: 2min 24s (started: 2024-08-01 20:59:58 +00:00)


In [ ]:
import csv
import json
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Function to get auditor assessment scores from OpenAI API
def get_auditor_assessment_scores(text):
    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an assistant helping to evaluate the performance of an auditor. Read the provided audit negotiation transcript where the auditor is referred to as 'auditor' and the client as 'client 1/2/3/4' or 'speaker 1/2/3/4/…'."
                    "Background: There are two major dimensions underlying individual variations in moral judgments. Individuals high on relativism reject the possibility of formulating or relying on universal moral rules when drawing conclusions about moral questions. Individuals high on idealism assume that desirable consequences can, with the 'right' action, always be obtained. "
                    "Your task: Evaluate the following statements on a scale of Completely Disagree - 1, Largely Disagree - 2, Moderately Disagree - 3, Slightly Disagree - 4, Neither Agree nor Disagree - 5, Slightly Agree - 6, Moderately Agree - 7, Largely Agree - 8, and Completely Agree - 9:"
                    "Statement 1. The auditor demonstrates high relativism by evaluating moral issues in the context of specific situations rather than following consistent moral rules."
                    "Statement 2. The auditor demonstrates high idealism by believing that desirable outcomes can always be achieved through the 'right' actions"
                    "Provide your 1 to 9 integer responses in JSON format as below:"
                    "{\"Relativism\": 0, \"Idealism\": 0}"
                )
            },
            {
                "role": "user",
                "content": text
            }
        ],
        temperature=0.7,
        max_tokens=100,
        top_p=1
    )

    json_object = json.loads(response.choices[0].message.content)
    return json_object

# List to store extracted values
data = []

# Iterate over each item in trimmed_text_list and get assessment scores
for text in trimmed_text_list:
    assessment_scores = get_auditor_assessment_scores(text)
    data.append([assessment_scores["Relativism"], assessment_scores["Idealism"]])
    print(f"Relativism: {assessment_scores['Relativism']}, Idealism: {assessment_scores['Idealism']}")

# Save data to CSV file
with open('/content/drive/MyDrive/EPQ & NEG/data/OpenAI API/auditor_epq_scores_0801v2.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Relativism', 'Idealism'])
    writer.writerows(data)

print("Data saved to auditor_epq_scores_0801v2.csv")

Relativism: 8, Idealism: 4
Relativism: 7, Idealism: 4
Relativism: 6, Idealism: 4
Relativism: 7, Idealism: 3
Relativism: 6, Idealism: 4
Relativism: 7, Idealism: 3
Relativism: 6, Idealism: 3
Relativism: 5, Idealism: 6
Relativism: 6, Idealism: 5
Relativism: 7, Idealism: 4
Relativism: 6, Idealism: 3
Relativism: 5, Idealism: 6
Relativism: 8, Idealism: 5
Relativism: 8, Idealism: 2
Relativism: 6, Idealism: 4
Relativism: 7, Idealism: 3
Relativism: 8, Idealism: 3
Relativism: 6, Idealism: 5
Relativism: 6, Idealism: 4
Relativism: 5, Idealism: 4
Relativism: 8, Idealism: 4
Relativism: 6, Idealism: 2
Data saved to auditor_epq_scores_0801v2.csv
time: 2min 58s (started: 2024-08-01 21:02:46 +00:00)


In [ ]:
import csv
import json
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Function to get auditor assessment scores from OpenAI API
def get_auditor_assessment_scores(text):
    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an assistant helping to evaluate the performance of an auditor. Read the provided audit negotiation transcript where the auditor is referred to as 'auditor' and the client as 'client 1/2/3/4' or 'speaker 1/2/3/4/…'."
                    "Background: There are two major dimensions underlying individual variations in moral judgments. Individuals high on relativism reject the possibility of formulating or relying on universal moral rules when drawing conclusions about moral questions. Individuals high on idealism assume that desirable consequences can, with the 'right' action, always be obtained. "
                    "Your task: Evaluate the following statements on a scale of Completely Disagree - 1, Largely Disagree - 2, Moderately Disagree - 3, Slightly Disagree - 4, Neither Agree nor Disagree - 5, Slightly Agree - 6, Moderately Agree - 7, Largely Agree - 8, and Completely Agree - 9:"
                    "Statement 1. The auditor demonstrates high relativism by evaluating moral issues in the context of specific situations rather than following consistent moral rules."
                    "Statement 2. The auditor demonstrates high idealism by believing that desirable outcomes can always be achieved through the 'right' actions"
                    "Provide your 1 to 9 integer responses in JSON format as below:"
                    "{\"Relativism\": 0, \"Idealism\": 0}"
                )
            },
            {
                "role": "user",
                "content": text
            }
        ],
        temperature=0.7,
        max_tokens=100,
        top_p=1
    )

    json_object = json.loads(response.choices[0].message.content)
    return json_object

# List to store extracted values
data = []

# Iterate over each item in trimmed_text_list and get assessment scores
for text in trimmed_text_list:
    assessment_scores = get_auditor_assessment_scores(text)
    data.append([assessment_scores["Relativism"], assessment_scores["Idealism"]])
    print(f"Relativism: {assessment_scores['Relativism']}, Idealism: {assessment_scores['Idealism']}")

# Save data to CSV file
with open('/link/to/a/folder/you/want/to/store/your/result/file/with/the/name/as/auditor_epq_scores_0801v3.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Relativism', 'Idealism'])
    writer.writerows(data)

print("Data saved to auditor_epq_scores_0801v3.csv")

Relativism: 8, Idealism: 3
Relativism: 8, Idealism: 5
Relativism: 6, Idealism: 7
Relativism: 7, Idealism: 4
Relativism: 6, Idealism: 5
Relativism: 6, Idealism: 3
Relativism: 6, Idealism: 5
Relativism: 7, Idealism: 6
Relativism: 5, Idealism: 4
Relativism: 4, Idealism: 3
Relativism: 7, Idealism: 4
Relativism: 8, Idealism: 7
Relativism: 8, Idealism: 2
Relativism: 6, Idealism: 5
Relativism: 5, Idealism: 3
Relativism: 5, Idealism: 6
Relativism: 4, Idealism: 6
Relativism: 7, Idealism: 8
Relativism: 4, Idealism: 6
Relativism: 6, Idealism: 4
Relativism: 4, Idealism: 2
Relativism: 6, Idealism: 3
Data saved to auditor_epq_scores_0801v3.csv
time: 3min 22s (started: 2024-08-01 21:05:45 +00:00)


### separate into Inv and AR

In [ ]:
import csv
import json
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Function to get auditor assessment scores from OpenAI API
def get_auditor_assessment_scores(text):
    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an assistant helping to evaluate the performance of an auditor. Read the provided audit negotiation transcript where the auditor is referred to as 'auditor' and the client as 'client 1/2/3/4' or 'speaker 1/2/3/4/…'."
                    "Background: There are two major dimensions underlying individual variations in moral judgments. Individuals high on relativism reject the possibility of formulating or relying on universal moral rules when drawing conclusions about moral questions. Individuals high on idealism assume that desirable consequences can, with the 'right' action, always be obtained. "
                    "Your first task (do NOT print results): Separate the negotiation transcript into two parts: one for inventory write-off, the other for the reduction in allowance for doubtful accounts of accounts receivable (AR). The auditor is simply called “auditor” and clients are called “client 1/2/3/4” or “speaker 1/2/3/4/…” in the transcript conversations. "
                    "Your second task (print results in the designated format): For each part, evaluate the following statements on a scale of Completely Disagree - 1, Largely Disagree - 2, Moderately Disagree - 3, Slightly Disagree - 4, Neither Agree nor Disagree - 5, Slightly Agree - 6, Moderately Agree - 7, Largely Agree - 8, and Completely Agree - 9:"
                    "Statement 1. The auditor demonstrates high relativism by evaluating moral issues in the context of specific situations rather than following consistent moral rules."
                    "Statement 2. The auditor demonstrates high idealism by believing that desirable outcomes can always be achieved through the 'right' actions"
                    "Provide your 1 to 9 integer responses from the second task in JSON format as below:"
                    "{\"Relativism_inventory\": 0, \"Idealism_inventory\": 0, \"Relativism_AR\": 0, \"Idealism_AR\": 0}"
                )
            },
            {
                "role": "user",
                "content": text
            }
        ],
        temperature=0.7,
        max_tokens=100,
        top_p=1
    )

    json_object = json.loads(response.choices[0].message.content)
    return json_object

# List to store extracted values
data = []

# Iterate over each item in trimmed_text_list and get assessment scores
for text in trimmed_text_list:
    assessment_scores = get_auditor_assessment_scores(text)
    data.append([assessment_scores["Relativism_inventory"], assessment_scores["Idealism_inventory"], assessment_scores["Relativism_AR"], assessment_scores["Idealism_AR"]])
    print(f"Relativism_inventory: {assessment_scores['Relativism_inventory']}, Idealism_inventory: {assessment_scores['Idealism_inventory']}, Relativism_AR: {assessment_scores['Relativism_AR']}, Idealism_AR: {assessment_scores['Idealism_AR']}")

# Save data to CSV file
with open('/link/to/a/folder/you/want/to/store/your/result/file/with/the/name/as/auditor_action_inv_ar_epq_scores_0806v1.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Relativism_inventory', 'Idealism_inventory', 'Relativism_AR', 'Idealism_AR'])
    writer.writerows(data)

print("Data saved.")

Relativism_inventory: 6, Idealism_inventory: 5, Relativism_AR: 5, Idealism_AR: 6
Relativism_inventory: 7, Idealism_inventory: 5, Relativism_AR: 8, Idealism_AR: 6
Relativism_inventory: 6, Idealism_inventory: 7, Relativism_AR: 8, Idealism_AR: 6
Relativism_inventory: 5, Idealism_inventory: 4, Relativism_AR: 6, Idealism_AR: 3
Relativism_inventory: 5, Idealism_inventory: 6, Relativism_AR: 4, Idealism_AR: 7
Relativism_inventory: 4, Idealism_inventory: 5, Relativism_AR: 6, Idealism_AR: 4
Relativism_inventory: 7, Idealism_inventory: 5, Relativism_AR: 6, Idealism_AR: 4
Relativism_inventory: 7, Idealism_inventory: 6, Relativism_AR: 5, Idealism_AR: 7
Relativism_inventory: 6, Idealism_inventory: 5, Relativism_AR: 4, Idealism_AR: 6
Relativism_inventory: 6, Idealism_inventory: 6, Relativism_AR: 5, Idealism_AR: 5
Relativism_inventory: 6, Idealism_inventory: 4, Relativism_AR: 7, Idealism_AR: 5
Relativism_inventory: 7, Idealism_inventory: 3, Relativism_AR: 8, Idealism_AR: 4
Relativism_inventory: 8, Ide

In [ ]:
import csv
import json
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Function to get auditor assessment scores from OpenAI API
def get_auditor_assessment_scores(text):
    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an assistant helping to evaluate the performance of an auditor. Read the provided audit negotiation transcript where the auditor is referred to as 'auditor' and the client as 'client 1/2/3/4' or 'speaker 1/2/3/4/…'."
                    "Background: There are two major dimensions underlying individual variations in moral judgments. Individuals high on relativism reject the possibility of formulating or relying on universal moral rules when drawing conclusions about moral questions. Individuals high on idealism assume that desirable consequences can, with the 'right' action, always be obtained. "
                    "Your first task (do NOT print results): Separate the negotiation transcript into two parts: one for inventory write-off, the other for the reduction in allowance for doubtful accounts of accounts receivable (AR). The auditor is simply called “auditor” and clients are called “client 1/2/3/4” or “speaker 1/2/3/4/…” in the transcript conversations. "
                    "Your second task (print results in the designated format): For each part, evaluate the following statements on a scale of Completely Disagree - 1, Largely Disagree - 2, Moderately Disagree - 3, Slightly Disagree - 4, Neither Agree nor Disagree - 5, Slightly Agree - 6, Moderately Agree - 7, Largely Agree - 8, and Completely Agree - 9:"
                    "Statement 1. The auditor demonstrates high relativism by evaluating moral issues in the context of specific situations rather than following consistent moral rules."
                    "Statement 2. The auditor demonstrates high idealism by believing that desirable outcomes can always be achieved through the 'right' actions"
                    "Provide your 1 to 9 integer responses from the second task in JSON format as below:"
                    "{\"Relativism_inventory\": 0, \"Idealism_inventory\": 0, \"Relativism_AR\": 0, \"Idealism_AR\": 0}"
                )
            },
            {
                "role": "user",
                "content": text
            }
        ],
        temperature=0.7,
        max_tokens=100,
        top_p=1
    )

    json_object = json.loads(response.choices[0].message.content)
    return json_object

# List to store extracted values
data = []

# Iterate over each item in trimmed_text_list and get assessment scores
for text in trimmed_text_list:
    assessment_scores = get_auditor_assessment_scores(text)
    data.append([assessment_scores["Relativism_inventory"], assessment_scores["Idealism_inventory"], assessment_scores["Relativism_AR"], assessment_scores["Idealism_AR"]])
    print(f"Relativism_inventory: {assessment_scores['Relativism_inventory']}, Idealism_inventory: {assessment_scores['Idealism_inventory']}, Relativism_AR: {assessment_scores['Relativism_AR']}, Idealism_AR: {assessment_scores['Idealism_AR']}")

# Save data to CSV file
with open('/link/to/a/folder/you/want/to/store/your/result/file/with/the/name/as/auditor_action_inv_ar_epq_scores_0806v2.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Relativism_inventory', 'Idealism_inventory', 'Relativism_AR', 'Idealism_AR'])
    writer.writerows(data)

print("Data saved.")

Relativism_inventory: 4, Idealism_inventory: 5, Relativism_AR: 3, Idealism_AR: 6
Relativism_inventory: 4, Idealism_inventory: 6, Relativism_AR: 5, Idealism_AR: 7
Relativism_inventory: 7, Idealism_inventory: 6, Relativism_AR: 8, Idealism_AR: 7
Relativism_inventory: 6, Idealism_inventory: 4, Relativism_AR: 5, Idealism_AR: 3
Relativism_inventory: 8, Idealism_inventory: 6, Relativism_AR: 7, Idealism_AR: 6
Relativism_inventory: 6, Idealism_inventory: 5, Relativism_AR: 7, Idealism_AR: 7
Relativism_inventory: 5, Idealism_inventory: 3, Relativism_AR: 6, Idealism_AR: 7
Relativism_inventory: 7, Idealism_inventory: 5, Relativism_AR: 6, Idealism_AR: 4
Relativism_inventory: 7, Idealism_inventory: 6, Relativism_AR: 8, Idealism_AR: 7
Relativism_inventory: 4, Idealism_inventory: 6, Relativism_AR: 3, Idealism_AR: 5
Relativism_inventory: 8, Idealism_inventory: 5, Relativism_AR: 8, Idealism_AR: 4
Relativism_inventory: 6, Idealism_inventory: 7, Relativism_AR: 5, Idealism_AR: 4
Relativism_inventory: 4, Ide

In [ ]:
import csv
import json
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Function to get auditor assessment scores from OpenAI API
def get_auditor_assessment_scores(text):
    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an assistant helping to evaluate the performance of an auditor. Read the provided audit negotiation transcript where the auditor is referred to as 'auditor' and the client as 'client 1/2/3/4' or 'speaker 1/2/3/4/…'."
                    "Background: There are two major dimensions underlying individual variations in moral judgments. Individuals high on relativism reject the possibility of formulating or relying on universal moral rules when drawing conclusions about moral questions. Individuals high on idealism assume that desirable consequences can, with the 'right' action, always be obtained. "
                    "Your first task (do NOT print results): Separate the negotiation transcript into two parts: one for inventory write-off, the other for the reduction in allowance for doubtful accounts of accounts receivable (AR). The auditor is simply called “auditor” and clients are called “client 1/2/3/4” or “speaker 1/2/3/4/…” in the transcript conversations. "
                    "Your second task (print results in the designated format): For each part, evaluate the following statements on a scale of Completely Disagree - 1, Largely Disagree - 2, Moderately Disagree - 3, Slightly Disagree - 4, Neither Agree nor Disagree - 5, Slightly Agree - 6, Moderately Agree - 7, Largely Agree - 8, and Completely Agree - 9:"
                    "Statement 1. The auditor demonstrates high relativism by evaluating moral issues in the context of specific situations rather than following consistent moral rules."
                    "Statement 2. The auditor demonstrates high idealism by believing that desirable outcomes can always be achieved through the 'right' actions"
                    "Provide your 1 to 9 integer responses from the second task in JSON format as below:"
                    "{\"Relativism_inventory\": 0, \"Idealism_inventory\": 0, \"Relativism_AR\": 0, \"Idealism_AR\": 0}"
                )
            },
            {
                "role": "user",
                "content": text
            }
        ],
        temperature=0.7,
        max_tokens=100,
        top_p=1
    )

    json_object = json.loads(response.choices[0].message.content)
    return json_object

# List to store extracted values
data = []

# Iterate over each item in trimmed_text_list and get assessment scores
for text in trimmed_text_list:
    assessment_scores = get_auditor_assessment_scores(text)
    data.append([assessment_scores["Relativism_inventory"], assessment_scores["Idealism_inventory"], assessment_scores["Relativism_AR"], assessment_scores["Idealism_AR"]])
    print(f"Relativism_inventory: {assessment_scores['Relativism_inventory']}, Idealism_inventory: {assessment_scores['Idealism_inventory']}, Relativism_AR: {assessment_scores['Relativism_AR']}, Idealism_AR: {assessment_scores['Idealism_AR']}")

# Save data to CSV file
with open('/link/to/a/folder/you/want/to/store/your/result/file/with/the/name/as/auditor_action_inv_ar_epq_scores_0806v3.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Relativism_inventory', 'Idealism_inventory', 'Relativism_AR', 'Idealism_AR'])
    writer.writerows(data)

print("Data saved.")

Relativism_inventory: 6, Idealism_inventory: 5, Relativism_AR: 6, Idealism_AR: 4
Relativism_inventory: 5, Idealism_inventory: 4, Relativism_AR: 6, Idealism_AR: 6
Relativism_inventory: 6, Idealism_inventory: 4, Relativism_AR: 5, Idealism_AR: 4
Relativism_inventory: 7, Idealism_inventory: 6, Relativism_AR: 6, Idealism_AR: 5
Relativism_inventory: 6, Idealism_inventory: 3, Relativism_AR: 7, Idealism_AR: 4
Relativism_inventory: 6, Idealism_inventory: 3, Relativism_AR: 5, Idealism_AR: 4
Relativism_inventory: 6, Idealism_inventory: 7, Relativism_AR: 5, Idealism_AR: 6
Relativism_inventory: 6, Idealism_inventory: 4, Relativism_AR: 7, Idealism_AR: 6
Relativism_inventory: 5, Idealism_inventory: 4, Relativism_AR: 7, Idealism_AR: 6
Relativism_inventory: 7, Idealism_inventory: 5, Relativism_AR: 6, Idealism_AR: 6
Relativism_inventory: 6, Idealism_inventory: 4, Relativism_AR: 7, Idealism_AR: 3
Relativism_inventory: 7, Idealism_inventory: 4, Relativism_AR: 6, Idealism_AR: 5
Relativism_inventory: 6, Ide

## word count for each part

In [ ]:
import csv
import json
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Function to get auditor assessment scores and word counts from OpenAI API
def get_auditor_assessment_scores(text):
    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an assistant helping to evaluate the performance of an auditor. Read the provided audit negotiation transcript."
                    "Your task (do NOT return text, only JSON): Separate the negotiation transcript into two parts without any overlap: one for inventory write-off, the other for the reduction in allowance for doubtful accounts of accounts receivable (AR). Calculate the word count for each part and provide the result in JSON format as below:"
                    "{\"Word_count_inventory\": 0, \"Word_count_AR\": 0}"
                )
            },
            {
                "role": "user",
                "content": text
            } # where the auditor is referred to as 'auditor' and the client as 'client 1/2/3/4' or 'speaker 1/2/3/4/…'
        ],
        temperature=0.7,
        max_tokens=100,
        top_p=1
    )

    json_object = json.loads(response.choices[0].message.content)
    return json_object

# List to store extracted values
data = []

# Iterate over each item in trimmed_text_list and get word counts
for text in trimmed_text_list:
    assessment_scores = get_auditor_assessment_scores(text)

    # Calculate total words and percentages
    total_words = assessment_scores["Word_count_inventory"] + assessment_scores["Word_count_AR"] # len(text.split())
    percent_words_inventory = (assessment_scores["Word_count_inventory"] / total_words) * 100 if total_words > 0 else 0
    percent_words_AR = (assessment_scores["Word_count_AR"] / total_words) * 100 if total_words > 0 else 0

    data.append([
        percent_words_inventory,
        percent_words_AR
    ])
    print(f"Word_count_inventory: {assessment_scores['Word_count_inventory']}, Word_count_AR: {assessment_scores['Word_count_AR']}, total_words: {total_words}, Percent_words_inventory: {percent_words_inventory}, Percent_words_AR: {percent_words_AR}")

# Save data to CSV file
with open('/link/to/a/folder/you/want/to/store/your/result/file/with/the/name/as/auditor_action_inv_ar_word_count_percent_0810.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Percent_words_inventory', 'Percent_words_AR'])
    writer.writerows(data)

print("Data saved.")


Word_count_inventory: 3588, Word_count_AR: 2252, total_words: 5840, Percent_words_inventory: 61.438356164383556, Percent_words_AR: 38.56164383561644
Word_count_inventory: 1629, Word_count_AR: 1344, total_words: 2973, Percent_words_inventory: 54.79313824419778, Percent_words_AR: 45.20686175580222
Word_count_inventory: 1384, Word_count_AR: 788, total_words: 2172, Percent_words_inventory: 63.72007366482505, Percent_words_AR: 36.279926335174956
Word_count_inventory: 504, Word_count_AR: 260, total_words: 764, Percent_words_inventory: 65.96858638743456, Percent_words_AR: 34.031413612565444
Word_count_inventory: 3722, Word_count_AR: 1221, total_words: 4943, Percent_words_inventory: 75.29840178029536, Percent_words_AR: 24.701598219704632
Word_count_inventory: 1023, Word_count_AR: 647, total_words: 1670, Percent_words_inventory: 61.25748502994012, Percent_words_AR: 38.74251497005988
Word_count_inventory: 1096, Word_count_AR: 535, total_words: 1631, Percent_words_inventory: 67.19803801348866, Pe

## Sentiment score

### loughran mcdonald dictionary

In [ ]:
import pandas as pd
import re

# Load the CSV file into a DataFrame
df = pd.read_csv('/content/drive/MyDrive/EPQ & NEG/data/Sentiment/Loughran-McDonald_MasterDictionary_1993-2023.csv')

# Filter positive words
positive_words = set(df[df['Positive'] > 0]['Word'])

# Filter negative words
negative_words = set(df[df['Negative'] > 0]['Word'])

def tone_score(text, positive_words, negative_words):
    # Preprocess the text: remove punctuation and convert to upper case
    text = re.sub(r'[^\w\s]', '', text).upper()

    # Tokenize the text into words
    words = text.split()
    print(words)
    # Count positive and negative words
    positive_count = sum(1 for word in words if word in positive_words)
    negative_count = sum(1 for word in words if word in negative_words)
    print(positive_count)
    print(negative_count)
    # Calculate tone score
    tone = (positive_count - negative_count) / len(words) if words else 0

    return tone

# Example usage
example_text = "This is an example string with some positive and negative words. Not abandon it."
score = tone_score(example_text, positive_words, negative_words)
print(f"Tone Score: {score}")


['THIS', 'IS', 'AN', 'EXAMPLE', 'STRING', 'WITH', 'SOME', 'POSITIVE', 'AND', 'NEGATIVE', 'WORDS', 'NOT', 'ABANDON', 'IT']
1
2
Tone Score: -0.07142857142857142
time: 204 ms (started: 2024-07-23 02:44:55 +00:00)


In [ ]:
import pandas as pd
import re
import csv
import nltk

# # Load the CSV file into a DataFrame
# df = pd.read_csv('/content/drive/MyDrive/EPQ & NEG/data/Sentiment/Loughran-McDonald_MasterDictionary_1993-2023.csv')

# # Filter positive words
# positive_words = set(df[df['Positive'] > 0]['Word'])

# # Filter negative words
# negative_words = set(df[df['Negative'] > 0]['Word'])

# Function to calculate the tone score and counts
def tone_score(text, positive_words, negative_words):
    # Preprocess the text: remove punctuation and convert to upper case
    text = re.sub(r'[^\w\s]', '', text).upper()

    # Tokenize the text into words
    words = text.split()

    # Count positive and negative words
    positive_count = sum(1 for word in words if word in positive_words)
    negative_count = sum(1 for word in words if word in negative_words)

    # Calculate tone score
    tone = (positive_count - negative_count) / len(words) if words else 0

    return positive_count, negative_count, len(words), tone

# Function to analyze sentiment for each speaker
def analyze_sentiment(transcript):
    segments = transcript.split('\n\n')
    text_by_speaker = {'Client': '', 'Auditor': ''}

    # Iterate through the segments in pairs
    for i in range(2, len(segments), 2):
        speaker_segment = segments[i-1].strip().lower()
        text_segment = segments[i].strip()

        if 'auditor' in speaker_segment:
            current_speaker = 'Auditor'
        else:
            current_speaker = 'Client'

        text_by_speaker[current_speaker] += ' ' + text_segment

    # Calculate sentiment details for each speaker
    sentiment_details = []
    for speaker, text in text_by_speaker.items():
        pos_count, neg_count, total_words, score = tone_score(text, positive_words, negative_words)
        sentiment_details.append((speaker, pos_count, neg_count, total_words, score))

    return sentiment_details

# # Example trimmed_text_list
# trimmed_text_list = [
#     "Auditor:\n\nThis is an auditor's part.\n\nClient:\n\nThis is a client's part.",
#     # Add more transcripts as needed
# ]

# Analyze sentiment for each transcript and save to CSV
with open('/content/drive/MyDrive/EPQ & NEG/data/Sentiment/sentiment_scores_loughran_mcdonald.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Part', 'Positive Words', 'Negative Words', 'Total Words', 'Tone Score'])  # Header row

    for transcript in trimmed_text_list:
        sentiment_details = analyze_sentiment(transcript)
        for detail in sentiment_details:
            writer.writerow(detail)

print("Sentiment scores saved to sentiment_scores.csv")

Sentiment scores saved to sentiment_scores.csv
time: 50.5 ms (started: 2024-07-23 03:05:20 +00:00)


# Load the planning memos

In [ ]:
import docx2txt
import os

text_list = []
directory = '/content/drive/MyDrive/EPQ & NEG/data/data collection steps & prompts/planning_text'

for filename in sorted(os.listdir(directory)): # sorted
  if filename.endswith(".docx"):
    filepath = os.path.join(directory, filename)
    text = docx2txt.process(filepath)
    text_list.append(text)

time: 10.6 s (started: 2024-08-09 03:25:30 +00:00)


## Professional skepticism questionnaire (three runs)

In [ ]:
import csv
import json
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Function to get auditor assessment scores from OpenAI API
def get_auditor_assessment_scores(text):
    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an assistant helping to evaluate the performance of an auditor based on the provided audit planning documentation."
                    "Background: Professional skepticism incorporates the attributes associated with being a skeptic to a professional setting that requires due diligence and a standard of care. Questioning mind is about maintaining an attitude of doubt and curiosity, and being prepared to challenge the status quo if the audit evidence suggests that there may be a problem. It’s about not taking things at face value, but digging deeper to ensure the financial statements present a true and fair view."
                    "Your task: Answer the following questions on a scale of 1 (strongly disagree), 2 (disagree), 3 (neither agree or disagree), 4 (agree), and 5 (strongly agree):"
                    "Q17. The auditor demonstrated professional skepticism while planning alternatives."
                    "Q18. While planning alternatives, the auditor's approach included a questioning mind."
                    "Q19. While planning alternatives, the auditor aimed to critically assess the available audit evidence."
                    "Q20. While planning alternatives, the auditor was alert to conditions which could indicate possible misstatement due to error or fraud."
                    "Provide your 1 to 5 integer responses in JSON format as below:"
                    "{\"Q17\": 0, \"Q18\": 0, \"Q19\": 0, \"Q20\": 0}"
                )
            },
            {
                "role": "user",
                "content": text
            }
        ],
        temperature=0.7,
        max_tokens=100,
        top_p=1
    )

    json_object = json.loads(response.choices[0].message.content)
    return json_object

# List to store extracted values
data = []

# Iterate over each item in trimmed_text_list and get assessment scores
for text in text_list:
    assessment_scores = get_auditor_assessment_scores(text)
    data.append([assessment_scores["Q17"], assessment_scores["Q18"], assessment_scores["Q19"], assessment_scores["Q20"]])
    print(f"Q17: {assessment_scores['Q17']}, Q18: {assessment_scores['Q18']}, Q19: {assessment_scores['Q19']}, Q20: {assessment_scores['Q20']}")

# Save data to CSV file
with open('/link/to/a/folder/you/want/to/store/your/result/file/with/the/name/as/auditor_planning_questionnaire_scores_0806v1.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Q17', 'Q18', 'Q19', 'Q20'])
    writer.writerows(data)

print("Data saved.")

Q17: 3, Q18: 3, Q19: 4, Q20: 3
Q17: 4, Q18: 4, Q19: 5, Q20: 5
Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 3, Q18: 4, Q19: 4, Q20: 4
Q17: 5, Q18: 5, Q19: 5, Q20: 5
Q17: 4, Q18: 4, Q19: 5, Q20: 5
Q17: 4, Q18: 4, Q19: 4, Q20: 5
Q17: 5, Q18: 5, Q19: 5, Q20: 5
Q17: 4, Q18: 5, Q19: 4, Q20: 4
Q17: 3, Q18: 3, Q19: 4, Q20: 4
Q17: 3, Q18: 4, Q19: 4, Q20: 4
Q17: 2, Q18: 3, Q19: 2, Q20: 2
Q17: 4, Q18: 4, Q19: 4, Q20: 5
Q17: 4, Q18: 5, Q19: 5, Q20: 5
Q17: 5, Q18: 5, Q19: 5, Q20: 5
Q17: 4, Q18: 4, Q19: 5, Q20: 4
Data saved.
time: 13.2 s (started: 2024-08-07 03:37:46 +00:00)


In [ ]:
import csv
import json
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Function to get auditor assessment scores from OpenAI API
def get_auditor_assessment_scores(text):
    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an assistant helping to evaluate the performance of an auditor based on the provided audit planning documentation."
                    "Background: Professional skepticism incorporates the attributes associated with being a skeptic to a professional setting that requires due diligence and a standard of care. Questioning mind is about maintaining an attitude of doubt and curiosity, and being prepared to challenge the status quo if the audit evidence suggests that there may be a problem. It’s about not taking things at face value, but digging deeper to ensure the financial statements present a true and fair view."
                    "Your task: Answer the following questions on a scale of 1 (strongly disagree), 2 (disagree), 3 (neither agree or disagree), 4 (agree), and 5 (strongly agree):"
                    "Q17. The auditor demonstrated professional skepticism while planning alternatives."
                    "Q18. While planning alternatives, the auditor's approach included a questioning mind."
                    "Q19. While planning alternatives, the auditor aimed to critically assess the available audit evidence."
                    "Q20. While planning alternatives, the auditor was alert to conditions which could indicate possible misstatement due to error or fraud."
                    "Provide your 1 to 5 integer responses in JSON format as below:"
                    "{\"Q17\": 0, \"Q18\": 0, \"Q19\": 0, \"Q20\": 0}"
                )
            },
            {
                "role": "user",
                "content": text
            }
        ],
        temperature=0.7,
        max_tokens=100,
        top_p=1
    )

    json_object = json.loads(response.choices[0].message.content)
    return json_object

# List to store extracted values
data = []

# Iterate over each item in trimmed_text_list and get assessment scores
for text in text_list:
    assessment_scores = get_auditor_assessment_scores(text)
    data.append([assessment_scores["Q17"], assessment_scores["Q18"], assessment_scores["Q19"], assessment_scores["Q20"]])
    print(f"Q17: {assessment_scores['Q17']}, Q18: {assessment_scores['Q18']}, Q19: {assessment_scores['Q19']}, Q20: {assessment_scores['Q20']}")

# Save data to CSV file
with open('/link/to/a/folder/you/want/to/store/your/result/file/with/the/name/as/auditor_planning_questionnaire_scores_0806v2.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Q17', 'Q18', 'Q19', 'Q20'])
    writer.writerows(data)

print("Data saved.")

Q17: 4, Q18: 4, Q19: 4, Q20: 3
Q17: 4, Q18: 4, Q19: 5, Q20: 5
Q17: 4, Q18: 4, Q19: 5, Q20: 5
Q17: 4, Q18: 5, Q19: 5, Q20: 4
Q17: 4, Q18: 4, Q19: 5, Q20: 5
Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 5, Q18: 5, Q19: 5, Q20: 5
Q17: 5, Q18: 5, Q19: 5, Q20: 5
Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 3, Q18: 4, Q19: 4, Q20: 3
Q17: 4, Q18: 5, Q19: 4, Q20: 5
Q17: 4, Q18: 4, Q19: 4, Q20: 5
Q17: 4, Q18: 5, Q19: 5, Q20: 5
Q17: 3, Q18: 3, Q19: 4, Q20: 4
Data saved.
time: 12.5 s (started: 2024-08-07 03:37:59 +00:00)


In [ ]:
import csv
import json
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Function to get auditor assessment scores from OpenAI API
def get_auditor_assessment_scores(text):
    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an assistant helping to evaluate the performance of an auditor based on the provided audit planning documentation."
                    "Background: Professional skepticism incorporates the attributes associated with being a skeptic to a professional setting that requires due diligence and a standard of care. Questioning mind is about maintaining an attitude of doubt and curiosity, and being prepared to challenge the status quo if the audit evidence suggests that there may be a problem. It’s about not taking things at face value, but digging deeper to ensure the financial statements present a true and fair view."
                    "Your task: Answer the following questions on a scale of 1 (strongly disagree), 2 (disagree), 3 (neither agree or disagree), 4 (agree), and 5 (strongly agree):"
                    "Q17. The auditor demonstrated professional skepticism while planning alternatives."
                    "Q18. While planning alternatives, the auditor's approach included a questioning mind."
                    "Q19. While planning alternatives, the auditor aimed to critically assess the available audit evidence."
                    "Q20. While planning alternatives, the auditor was alert to conditions which could indicate possible misstatement due to error or fraud."
                    "Provide your 1 to 5 integer responses in JSON format as below:"
                    "{\"Q17\": 0, \"Q18\": 0, \"Q19\": 0, \"Q20\": 0}"
                )
            },
            {
                "role": "user",
                "content": text
            }
        ],
        temperature=0.7,
        max_tokens=100,
        top_p=1
    )

    json_object = json.loads(response.choices[0].message.content)
    return json_object

# List to store extracted values
data = []

# Iterate over each item in trimmed_text_list and get assessment scores
for text in text_list:
    assessment_scores = get_auditor_assessment_scores(text)
    data.append([assessment_scores["Q17"], assessment_scores["Q18"], assessment_scores["Q19"], assessment_scores["Q20"]])
    print(f"Q17: {assessment_scores['Q17']}, Q18: {assessment_scores['Q18']}, Q19: {assessment_scores['Q19']}, Q20: {assessment_scores['Q20']}")

# Save data to CSV file
with open('/link/to/a/folder/you/want/to/store/your/result/file/with/the/name/as/auditor_planning_questionnaire_scores_0806v3.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Q17', 'Q18', 'Q19', 'Q20'])
    writer.writerows(data)

print("Data saved.")

Q17: 3, Q18: 2, Q19: 3, Q20: 3
Q17: 4, Q18: 5, Q19: 4, Q20: 4
Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 3, Q18: 4, Q19: 4, Q20: 4
Q17: 5, Q18: 5, Q19: 5, Q20: 5
Q17: 4, Q18: 4, Q19: 5, Q20: 5
Q17: 5, Q18: 5, Q19: 5, Q20: 5
Q17: 5, Q18: 5, Q19: 5, Q20: 5
Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 4, Q18: 4, Q19: 4, Q20: 3
Q17: 3, Q18: 3, Q19: 3, Q20: 3
Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 4, Q18: 4, Q19: 5, Q20: 5
Q17: 4, Q18: 4, Q19: 5, Q20: 5
Q17: 3, Q18: 4, Q19: 4, Q20: 4
Q17: 4, Q18: 5, Q19: 4, Q20: 5
Data saved.
time: 13.1 s (started: 2024-08-07 03:38:11 +00:00)


### separate into Inv and AR (three runs)

In [ ]:
import csv
import json
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Function to get auditor assessment scores from OpenAI API
def get_auditor_assessment_scores(text):
    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an assistant helping to evaluate the performance of an auditor based on the provided audit planning documentation."
                    "Background: Professional skepticism incorporates the attributes associated with being a skeptic to a professional setting that requires due diligence and a standard of care. Questioning mind is about maintaining an attitude of doubt and curiosity, and being prepared to challenge the status quo if the audit evidence suggests that there may be a problem. It’s about not taking things at face value, but digging deeper to ensure the financial statements present a true and fair view."
                    "Your first task (do NOT print results): Separate the negotiation transcript into two parts: one for inventory write-off, the other for the reduction in allowance for doubtful accounts of accounts receivable (AR)."
                    "Your second task (print results in the designated format): For each part, answer the following questions on a scale of 1 (strongly disagree), 2 (disagree), 3 (neither agree or disagree), 4 (agree), and 5 (strongly agree). If the two parts cannot be separated, use the same responses for both the Inventory and AR questions, based on the combined content of the two sections."
                    "Q17. The auditor demonstrated professional skepticism while planning alternatives."
                    "Q18. While planning alternatives, the auditor's approach included a questioning mind."
                    "Q19. While planning alternatives, the auditor aimed to critically assess the available audit evidence."
                    "Q20. While planning alternatives, the auditor was alert to conditions which could indicate possible misstatement due to error or fraud."
                    "Provide your 1 to 5 integer responses from the second task in JSON format as below:"
                    "{\"Q17_inventory\": 0, \"Q18_inventory\": 0, \"Q19_inventory\": 0, \"Q20_inventory\": 0, \"Q17_AR\": 0, \"Q18_AR\": 0, \"Q19_AR\": 0, \"Q20_AR\": 0, }"
                )
            },
            {
                "role": "user",
                "content": text
            }
        ],
        temperature=0.7,
        max_tokens=100,
        top_p=1
    )

    json_object = json.loads(response.choices[0].message.content)
    return json_object

# List to store extracted values
data = []

# Iterate over each item in trimmed_text_list and get assessment scores
for text in text_list:
    assessment_scores = get_auditor_assessment_scores(text)
    data.append([assessment_scores["Q17_inventory"], assessment_scores["Q18_inventory"], assessment_scores["Q19_inventory"], assessment_scores["Q20_inventory"], assessment_scores["Q17_AR"], assessment_scores["Q18_AR"], assessment_scores["Q19_AR"], assessment_scores["Q20_AR"]])
    print(f"Q17: {assessment_scores['Q17_inventory']}, Q18: {assessment_scores['Q18_inventory']}, Q19: {assessment_scores['Q19_inventory']}, Q20: {assessment_scores['Q20_inventory']}, Q17: {assessment_scores['Q17_AR']}, Q18: {assessment_scores['Q18_AR']}, Q19: {assessment_scores['Q19_AR']}, Q20: {assessment_scores['Q20_AR']}")

# Save data to CSV file
with open('/link/to/a/folder/you/want/to/store/your/result/file/with/the/name/as/auditor_planning_inv_ar_questionnaire_scores_0806v1.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Q17_inventory', 'Q18_inventory', 'Q19_inventory', 'Q20_inventory', 'Q17_AR', 'Q18_AR', 'Q19_AR', 'Q20_AR'])
    writer.writerows(data)

print("Data saved.")

Q17: 2, Q18: 2, Q19: 2, Q20: 2, Q17: 2, Q18: 2, Q19: 2, Q20: 2
Q17: 3, Q18: 3, Q19: 4, Q20: 3, Q17: 3, Q18: 3, Q19: 4, Q20: 3
Q17: 1, Q18: 1, Q19: 1, Q20: 1, Q17: 1, Q18: 1, Q19: 1, Q20: 1
Q17: 2, Q18: 2, Q19: 2, Q20: 2, Q17: 2, Q18: 2, Q19: 2, Q20: 2
Q17: 4, Q18: 4, Q19: 4, Q20: 4, Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 4, Q18: 4, Q19: 4, Q20: 4, Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 5, Q18: 5, Q19: 5, Q20: 5, Q17: 5, Q18: 5, Q19: 5, Q20: 5
Q17: 4, Q18: 4, Q19: 5, Q20: 5, Q17: 4, Q18: 5, Q19: 5, Q20: 4
Q17: 3, Q18: 3, Q19: 4, Q20: 4, Q17: 3, Q18: 3, Q19: 4, Q20: 4
Q17: 3, Q18: 3, Q19: 3, Q20: 3, Q17: 3, Q18: 3, Q19: 3, Q20: 3
Q17: 2, Q18: 2, Q19: 3, Q20: 2, Q17: 2, Q18: 2, Q19: 3, Q20: 2
Q17: 3, Q18: 3, Q19: 3, Q20: 3, Q17: 3, Q18: 3, Q19: 3, Q20: 3
Q17: 4, Q18: 4, Q19: 4, Q20: 4, Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 4, Q18: 4, Q19: 5, Q20: 5, Q17: 4, Q18: 4, Q19: 5, Q20: 5
Q17: 4, Q18: 4, Q19: 4, Q20: 4, Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 4, Q18: 5, Q19: 5, Q20: 4, Q17: 3, Q18: 4, Q19: 4,

In [ ]:
import csv
import json
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Function to get auditor assessment scores from OpenAI API
def get_auditor_assessment_scores(text):
    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an assistant helping to evaluate the performance of an auditor based on the provided audit planning documentation."
                    "Background: Professional skepticism incorporates the attributes associated with being a skeptic to a professional setting that requires due diligence and a standard of care. Questioning mind is about maintaining an attitude of doubt and curiosity, and being prepared to challenge the status quo if the audit evidence suggests that there may be a problem. It’s about not taking things at face value, but digging deeper to ensure the financial statements present a true and fair view."
                    "Your first task (do NOT print results): Separate the negotiation transcript into two parts: one for inventory write-off, the other for the reduction in allowance for doubtful accounts of accounts receivable (AR). Your second task (print results in the designated format): For each part, answer the following questions on a scale of 1 (strongly disagree), 2 (disagree), 3 (neither agree or disagree), 4 (agree), and 5 (strongly agree). If the two parts cannot be separated, use the same responses for both the Inventory and AR questions, based on the combined content of the two sections."
                    "Q17. The auditor demonstrated professional skepticism while planning alternatives."
                    "Q18. While planning alternatives, the auditor's approach included a questioning mind."
                    "Q19. While planning alternatives, the auditor aimed to critically assess the available audit evidence."
                    "Q20. While planning alternatives, the auditor was alert to conditions which could indicate possible misstatement due to error or fraud."
                    "Provide your 1 to 5 integer responses from the second task in JSON format as below:"
                    "{\"Q17_inventory\": 0, \"Q18_inventory\": 0, \"Q19_inventory\": 0, \"Q20_inventory\": 0, \"Q17_AR\": 0, \"Q18_AR\": 0, \"Q19_AR\": 0, \"Q20_AR\": 0, }"
                )
            },
            {
                "role": "user",
                "content": text
            }
        ],
        temperature=0.7,
        max_tokens=100,
        top_p=1
    )

    json_object = json.loads(response.choices[0].message.content)
    return json_object

# List to store extracted values
data = []

# Iterate over each item in trimmed_text_list and get assessment scores
for text in text_list:
    assessment_scores = get_auditor_assessment_scores(text)
    data.append([assessment_scores["Q17_inventory"], assessment_scores["Q18_inventory"], assessment_scores["Q19_inventory"], assessment_scores["Q20_inventory"], assessment_scores["Q17_AR"], assessment_scores["Q18_AR"], assessment_scores["Q19_AR"], assessment_scores["Q20_AR"]])
    print(f"Q17: {assessment_scores['Q17_inventory']}, Q18: {assessment_scores['Q18_inventory']}, Q19: {assessment_scores['Q19_inventory']}, Q20: {assessment_scores['Q20_inventory']}, Q17: {assessment_scores['Q17_AR']}, Q18: {assessment_scores['Q18_AR']}, Q19: {assessment_scores['Q19_AR']}, Q20: {assessment_scores['Q20_AR']}")

# Save data to CSV file
with open('/link/to/a/folder/you/want/to/store/your/result/file/with/the/name/as/auditor_planning_inv_ar_questionnaire_scores_0806v2.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Q17_inventory', 'Q18_inventory', 'Q19_inventory', 'Q20_inventory', 'Q17_AR', 'Q18_AR', 'Q19_AR', 'Q20_AR'])
    writer.writerows(data)

print("Data saved.")

Q17: 3, Q18: 3, Q19: 3, Q20: 3, Q17: 3, Q18: 3, Q19: 3, Q20: 3
Q17: 4, Q18: 4, Q19: 4, Q20: 4, Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 2, Q18: 3, Q19: 2, Q20: 3, Q17: 2, Q18: 3, Q19: 2, Q20: 3
Q17: 3, Q18: 3, Q19: 3, Q20: 3, Q17: 3, Q18: 3, Q19: 3, Q20: 3
Q17: 3, Q18: 4, Q19: 4, Q20: 3, Q17: 3, Q18: 4, Q19: 4, Q20: 3
Q17: 4, Q18: 4, Q19: 4, Q20: 4, Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 4, Q18: 4, Q19: 4, Q20: 5, Q17: 4, Q18: 4, Q19: 4, Q20: 5
Q17: 5, Q18: 5, Q19: 5, Q20: 5, Q17: 5, Q18: 5, Q19: 5, Q20: 5
Q17: 4, Q18: 4, Q19: 4, Q20: 4, Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 3, Q18: 3, Q19: 3, Q20: 3, Q17: 3, Q18: 3, Q19: 3, Q20: 3
Q17: 3, Q18: 4, Q19: 3, Q20: 3, Q17: 3, Q18: 4, Q19: 3, Q20: 3
Q17: 3, Q18: 3, Q19: 2, Q20: 2, Q17: 3, Q18: 3, Q19: 2, Q20: 2
Q17: 4, Q18: 4, Q19: 4, Q20: 5, Q17: 4, Q18: 4, Q19: 4, Q20: 5
Q17: 4, Q18: 4, Q19: 4, Q20: 4, Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 4, Q18: 4, Q19: 5, Q20: 5, Q17: 4, Q18: 4, Q19: 5, Q20: 5
Q17: 4, Q18: 4, Q19: 4, Q20: 4, Q17: 3, Q18: 3, Q19: 3,

In [ ]:
import csv
import json
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Function to get auditor assessment scores from OpenAI API
def get_auditor_assessment_scores(text):
    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an assistant helping to evaluate the performance of an auditor based on the provided audit planning documentation."
                    "Background: Professional skepticism incorporates the attributes associated with being a skeptic to a professional setting that requires due diligence and a standard of care. Questioning mind is about maintaining an attitude of doubt and curiosity, and being prepared to challenge the status quo if the audit evidence suggests that there may be a problem. It’s about not taking things at face value, but digging deeper to ensure the financial statements present a true and fair view."
                    "Your first task (do NOT print results): Separate the negotiation transcript into two parts: one for inventory write-off, the other for the reduction in allowance for doubtful accounts of accounts receivable (AR). Your second task (print results in the designated format): For each part, answer the following questions on a scale of 1 (strongly disagree), 2 (disagree), 3 (neither agree or disagree), 4 (agree), and 5 (strongly agree). If the two parts cannot be separated, use the same responses for both the Inventory and AR questions, based on the combined content of the two sections."
                    "Q17. The auditor demonstrated professional skepticism while planning alternatives."
                    "Q18. While planning alternatives, the auditor's approach included a questioning mind."
                    "Q19. While planning alternatives, the auditor aimed to critically assess the available audit evidence."
                    "Q20. While planning alternatives, the auditor was alert to conditions which could indicate possible misstatement due to error or fraud."
                    "Provide your 1 to 5 integer responses from the second task in JSON format as below:"
                    "{\"Q17_inventory\": 0, \"Q18_inventory\": 0, \"Q19_inventory\": 0, \"Q20_inventory\": 0, \"Q17_AR\": 0, \"Q18_AR\": 0, \"Q19_AR\": 0, \"Q20_AR\": 0, }"
                )
            },
            {
                "role": "user",
                "content": text
            }
        ],
        temperature=0.7,
        max_tokens=100,
        top_p=1
    )

    json_object = json.loads(response.choices[0].message.content)
    return json_object

# List to store extracted values
data = []

# Iterate over each item in trimmed_text_list and get assessment scores
for text in text_list:
    assessment_scores = get_auditor_assessment_scores(text)
    data.append([assessment_scores["Q17_inventory"], assessment_scores["Q18_inventory"], assessment_scores["Q19_inventory"], assessment_scores["Q20_inventory"], assessment_scores["Q17_AR"], assessment_scores["Q18_AR"], assessment_scores["Q19_AR"], assessment_scores["Q20_AR"]])
    print(f"Q17: {assessment_scores['Q17_inventory']}, Q18: {assessment_scores['Q18_inventory']}, Q19: {assessment_scores['Q19_inventory']}, Q20: {assessment_scores['Q20_inventory']}, Q17: {assessment_scores['Q17_AR']}, Q18: {assessment_scores['Q18_AR']}, Q19: {assessment_scores['Q19_AR']}, Q20: {assessment_scores['Q20_AR']}")

# Save data to CSV file
with open('/link/to/a/folder/you/want/to/store/your/result/file/with/the/name/as/auditor_planning_inv_ar_questionnaire_scores_0806v3.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Q17_inventory', 'Q18_inventory', 'Q19_inventory', 'Q20_inventory', 'Q17_AR', 'Q18_AR', 'Q19_AR', 'Q20_AR'])
    writer.writerows(data)

print("Data saved.")

Q17: 2, Q18: 2, Q19: 2, Q20: 2, Q17: 2, Q18: 2, Q19: 2, Q20: 2
Q17: 3, Q18: 3, Q19: 3, Q20: 3, Q17: 3, Q18: 3, Q19: 3, Q20: 3
Q17: 2, Q18: 2, Q19: 2, Q20: 2, Q17: 2, Q18: 2, Q19: 2, Q20: 2
Q17: 3, Q18: 3, Q19: 3, Q20: 3, Q17: 3, Q18: 3, Q19: 3, Q20: 3
Q17: 4, Q18: 4, Q19: 4, Q20: 4, Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 5, Q18: 5, Q19: 5, Q20: 5, Q17: 5, Q18: 5, Q19: 5, Q20: 5
Q17: 4, Q18: 5, Q19: 5, Q20: 4, Q17: 3, Q18: 4, Q19: 4, Q20: 4
Q17: 4, Q18: 4, Q19: 4, Q20: 5, Q17: 4, Q18: 4, Q19: 4, Q20: 5
Q17: 4, Q18: 4, Q19: 4, Q20: 4, Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 3, Q18: 3, Q19: 3, Q20: 3, Q17: 3, Q18: 3, Q19: 3, Q20: 3
Q17: 3, Q18: 3, Q19: 3, Q20: 3, Q17: 3, Q18: 3, Q19: 3, Q20: 3
Q17: 3, Q18: 3, Q19: 3, Q20: 3, Q17: 3, Q18: 3, Q19: 3, Q20: 3
Q17: 4, Q18: 4, Q19: 4, Q20: 4, Q17: 4, Q18: 4, Q19: 4, Q20: 4
Q17: 4, Q18: 4, Q19: 5, Q20: 5, Q17: 4, Q18: 4, Q19: 5, Q20: 5
Q17: 5, Q18: 5, Q19: 5, Q20: 5, Q17: 5, Q18: 5, Q19: 5, Q20: 5
Q17: 4, Q18: 4, Q19: 4, Q20: 4, Q17: 3, Q18: 3, Q19: 3,

## Ethicality

In [ ]:
import csv
import json
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Function to get auditor assessment scores from OpenAI API
def get_auditor_assessment_scores(text):
    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an assistant helping to evaluate the performance of an auditor based on the provided audit planning documentation."
                    "Background: There are two major dimensions underlying individual variations in moral judgments. Individuals high on relativism reject the possibility of formulating or relying on universal moral rules when drawing conclusions about moral questions. Individuals high on idealism assume that desirable consequences can, with the 'right' action, always be obtained. "
                    "Your task: Evaluate the following statements on a scale of Completely Disagree - 1, Largely Disagree - 2, Moderately Disagree - 3, Slightly Disagree - 4, Neither Agree nor Disagree - 5, Slightly Agree - 6, Moderately Agree - 7, Largely Agree - 8, and Completely Agree - 9:"
                    "Statement 1. The auditor demonstrates high relativism by evaluating moral issues in the context of specific situations rather than following consistent moral rules."
                    "Statement 2. The auditor demonstrates high idealism by believing that desirable outcomes can always be achieved through the 'right' actions"
                    "Provide your 1 to 9 integer responses in JSON format as below:"
                    "{\"Relativism\": 0, \"Idealism\": 0}"
                )
            },
            {
                "role": "user",
                "content": text
            }
        ],
        temperature=0.7,
        max_tokens=100,
        top_p=1
    )

    json_object = json.loads(response.choices[0].message.content)
    return json_object

# List to store extracted values
data = []

# Iterate over each item in trimmed_text_list and get assessment scores
for text in text_list:
    assessment_scores = get_auditor_assessment_scores(text)
    data.append([assessment_scores["Relativism"], assessment_scores["Idealism"]])
    print(f"Relativism: {assessment_scores['Relativism']}, Idealism: {assessment_scores['Idealism']}")

# Save data to CSV file
with open('/link/to/a/folder/you/want/to/store/your/result/file/with/the/name/as/auditor_epq_scores_planning_0802.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Relativism', 'Idealism'])
    writer.writerows(data)

print("Data saved to auditor_epq_scores_planning_0802.csv")

Relativism: 6, Idealism: 4
Relativism: 7, Idealism: 8
Relativism: 7, Idealism: 4
Relativism: 7, Idealism: 6
Relativism: 4, Idealism: 7
Relativism: 7, Idealism: 6
Relativism: 8, Idealism: 7
Relativism: 7, Idealism: 6
Relativism: 8, Idealism: 6
Relativism: 7, Idealism: 8
Relativism: 6, Idealism: 7
Relativism: 9, Idealism: 8
Relativism: 8, Idealism: 7
Relativism: 8, Idealism: 7
Relativism: 7, Idealism: 6
Relativism: 8, Idealism: 4
Data saved to auditor_epq_scores_planning_0802.csv
time: 14.8 s (started: 2024-08-02 14:32:25 +00:00)


In [ ]:
import csv
import json
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Function to get auditor assessment scores from OpenAI API
def get_auditor_assessment_scores(text):
    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an assistant helping to evaluate the performance of an auditor based on the provided audit planning documentation."
                    "Background: There are two major dimensions underlying individual variations in moral judgments. Individuals high on relativism reject the possibility of formulating or relying on universal moral rules when drawing conclusions about moral questions. Individuals high on idealism assume that desirable consequences can, with the 'right' action, always be obtained. "
                    "Your task: Evaluate the following statements on a scale of Completely Disagree - 1, Largely Disagree - 2, Moderately Disagree - 3, Slightly Disagree - 4, Neither Agree nor Disagree - 5, Slightly Agree - 6, Moderately Agree - 7, Largely Agree - 8, and Completely Agree - 9:"
                    "Statement 1. The auditor demonstrates high relativism by evaluating moral issues in the context of specific situations rather than following consistent moral rules."
                    "Statement 2. The auditor demonstrates high idealism by believing that desirable outcomes can always be achieved through the 'right' actions"
                    "Provide your 1 to 9 integer responses in JSON format as below:"
                    "{\"Relativism\": 0, \"Idealism\": 0}"
                )
            },
            {
                "role": "user",
                "content": text
            }
        ],
        temperature=0.7,
        max_tokens=100,
        top_p=1
    )

    json_object = json.loads(response.choices[0].message.content)
    return json_object

# List to store extracted values
data = []

# Iterate over each item in trimmed_text_list and get assessment scores
for text in text_list:
    assessment_scores = get_auditor_assessment_scores(text)
    data.append([assessment_scores["Relativism"], assessment_scores["Idealism"]])
    print(f"Relativism: {assessment_scores['Relativism']}, Idealism: {assessment_scores['Idealism']}")

# Save data to CSV file
with open('/content/drive/MyDrive/EPQ & NEG/data/OpenAI API/auditor_epq_scores_planning_0802v2.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Relativism', 'Idealism'])
    writer.writerows(data)

print("Data saved to auditor_epq_scores_planning_0802v2.csv")

Relativism: 4, Idealism: 6
Relativism: 8, Idealism: 7
Relativism: 4, Idealism: 3
Relativism: 6, Idealism: 7
Relativism: 4, Idealism: 7
Relativism: 4, Idealism: 6
Relativism: 7, Idealism: 6
Relativism: 8, Idealism: 7
Relativism: 7, Idealism: 5
Relativism: 4, Idealism: 6
Relativism: 5, Idealism: 6
Relativism: 6, Idealism: 7
Relativism: 7, Idealism: 7
Relativism: 8, Idealism: 7
Relativism: 7, Idealism: 5
Relativism: 8, Idealism: 5
Data saved to auditor_epq_scores_planning_0802v2.csv
time: 14.1 s (started: 2024-08-02 14:32:53 +00:00)


In [ ]:
import csv
import json
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Function to get auditor assessment scores from OpenAI API
def get_auditor_assessment_scores(text):
    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an assistant helping to evaluate the performance of an auditor based on the provided audit planning documentation."
                    "Background: There are two major dimensions underlying individual variations in moral judgments. Individuals high on relativism reject the possibility of formulating or relying on universal moral rules when drawing conclusions about moral questions. Individuals high on idealism assume that desirable consequences can, with the 'right' action, always be obtained. "
                    "Your task: Evaluate the following statements on a scale of Completely Disagree - 1, Largely Disagree - 2, Moderately Disagree - 3, Slightly Disagree - 4, Neither Agree nor Disagree - 5, Slightly Agree - 6, Moderately Agree - 7, Largely Agree - 8, and Completely Agree - 9:"
                    "Statement 1. The auditor demonstrates high relativism by evaluating moral issues in the context of specific situations rather than following consistent moral rules."
                    "Statement 2. The auditor demonstrates high idealism by believing that desirable outcomes can always be achieved through the 'right' actions"
                    "Provide your 1 to 9 integer responses in JSON format as below:"
                    "{\"Relativism\": 0, \"Idealism\": 0}"
                )
            },
            {
                "role": "user",
                "content": text
            }
        ],
        temperature=0.7,
        max_tokens=100,
        top_p=1
    )

    json_object = json.loads(response.choices[0].message.content)
    return json_object

# List to store extracted values
data = []

# Iterate over each item in trimmed_text_list and get assessment scores
for text in text_list:
    assessment_scores = get_auditor_assessment_scores(text)
    data.append([assessment_scores["Relativism"], assessment_scores["Idealism"]])
    print(f"Relativism: {assessment_scores['Relativism']}, Idealism: {assessment_scores['Idealism']}")

# Save data to CSV file
with open('/content/drive/MyDrive/EPQ & NEG/data/OpenAI API/auditor_epq_scores_planning_0802v3.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Relativism', 'Idealism'])
    writer.writerows(data)

print("Data saved to auditor_epq_scores_planning_0802v3.csv")

Relativism: 7, Idealism: 6
Relativism: 7, Idealism: 8
Relativism: 4, Idealism: 7
Relativism: 8, Idealism: 6
Relativism: 4, Idealism: 8
Relativism: 5, Idealism: 7
Relativism: 7, Idealism: 6
Relativism: 8, Idealism: 6
Relativism: 7, Idealism: 6
Relativism: 7, Idealism: 6
Relativism: 5, Idealism: 7
Relativism: 7, Idealism: 6
Relativism: 8, Idealism: 7
Relativism: 8, Idealism: 7
Relativism: 6, Idealism: 7
Relativism: 8, Idealism: 5
Data saved to auditor_epq_scores_planning_0802v3.csv
time: 13.4 s (started: 2024-08-02 14:33:24 +00:00)


### separate into Inv and AR 0806

In [ ]:
import csv
import json
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Function to get auditor assessment scores from OpenAI API
def get_auditor_assessment_scores(text):
    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an assistant helping to evaluate the performance of an auditor based on the provided audit planning documentation."
                    "Background: There are two major dimensions underlying individual variations in moral judgments. Individuals high on relativism reject the possibility of formulating or relying on universal moral rules when drawing conclusions about moral questions. Individuals high on idealism assume that desirable consequences can, with the 'right' action, always be obtained. "
                    "Your first task (do NOT print results): Separate the negotiation transcript into two parts: one for inventory write-off, the other for the reduction in allowance for doubtful accounts of accounts receivable (AR). Your second task (print results in the designated format): For each part, answer the following questions on a scale of Completely Disagree - 1, Largely Disagree - 2, Moderately Disagree - 3, Slightly Disagree - 4, Neither Agree nor Disagree - 5, Slightly Agree - 6, Moderately Agree - 7, Largely Agree - 8, and Completely Agree - 9. If the two parts cannot be separated, use the same responses for both the Inventory and AR questions, based on the combined content of the two sections."
                    "Statement 1. The auditor demonstrates high relativism by evaluating moral issues in the context of specific situations rather than following consistent moral rules."
                    "Statement 2. The auditor demonstrates high idealism by believing that desirable outcomes can always be achieved through the 'right' actions"
                    "Provide your 1 to 9 integer responses from the second task in JSON format as below:"
                    "{\"Relativism_inventory\": 0, \"Idealism_inventory\": 0, \"Relativism_AR\": 0, \"Idealism_AR\": 0}"
                )
            },
            {
                "role": "user",
                "content": text
            }
        ],
        temperature=0.7,
        max_tokens=100,
        top_p=1
    )

    json_object = json.loads(response.choices[0].message.content)
    return json_object

# List to store extracted values
data = []

# Iterate over each item in trimmed_text_list and get assessment scores
for text in text_list:
    assessment_scores = get_auditor_assessment_scores(text)
    data.append([assessment_scores["Relativism_inventory"], assessment_scores["Idealism_inventory"], assessment_scores["Relativism_AR"], assessment_scores["Idealism_AR"]])
    print(f"Relativism_inventory: {assessment_scores['Relativism_inventory']}, Idealism_inventory: {assessment_scores['Idealism_inventory']}, Relativism_AR: {assessment_scores['Relativism_AR']}, Idealism_AR: {assessment_scores['Idealism_AR']}")

# Save data to CSV file
with open('/content/drive/MyDrive/EPQ & NEG/data/OpenAI API/auditor_planning_inv_ar_epq_scores_0808v1.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Relativism_inventory', 'Idealism_inventory', 'Relativism_AR', 'Idealism_AR'])
    writer.writerows(data)

print("Data saved.")

Relativism_inventory: 2, Idealism_inventory: 4, Relativism_AR: 2, Idealism_AR: 4
Relativism_inventory: 6, Idealism_inventory: 7, Relativism_AR: 6, Idealism_AR: 7
Relativism_inventory: 1, Idealism_inventory: 2, Relativism_AR: 1, Idealism_AR: 2
Relativism_inventory: 2, Idealism_inventory: 6, Relativism_AR: 2, Idealism_AR: 6
Relativism_inventory: 3, Idealism_inventory: 7, Relativism_AR: 3, Idealism_AR: 7
Relativism_inventory: 6, Idealism_inventory: 7, Relativism_AR: 6, Idealism_AR: 7
Relativism_inventory: 7, Idealism_inventory: 6, Relativism_AR: 7, Idealism_AR: 6
Relativism_inventory: 8, Idealism_inventory: 3, Relativism_AR: 7, Idealism_AR: 4
Relativism_inventory: 6, Idealism_inventory: 5, Relativism_AR: 6, Idealism_AR: 5
Relativism_inventory: 7, Idealism_inventory: 6, Relativism_AR: 7, Idealism_AR: 6
Relativism_inventory: 6, Idealism_inventory: 7, Relativism_AR: 6, Idealism_AR: 7
Relativism_inventory: 3, Idealism_inventory: 7, Relativism_AR: 3, Idealism_AR: 7
Relativism_inventory: 8, Ide

In [ ]:
import csv
import json
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Function to get auditor assessment scores from OpenAI API
def get_auditor_assessment_scores(text):
    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an assistant helping to evaluate the performance of an auditor based on the provided audit planning documentation."
                    "Background: There are two major dimensions underlying individual variations in moral judgments. Individuals high on relativism reject the possibility of formulating or relying on universal moral rules when drawing conclusions about moral questions. Individuals high on idealism assume that desirable consequences can, with the 'right' action, always be obtained. "
                    "Your first task (do NOT print results): Separate the negotiation transcript into two parts: one for inventory write-off, the other for the reduction in allowance for doubtful accounts of accounts receivable (AR). Your second task (print results in the designated format): For each part, answer the following questions on a scale of Completely Disagree - 1, Largely Disagree - 2, Moderately Disagree - 3, Slightly Disagree - 4, Neither Agree nor Disagree - 5, Slightly Agree - 6, Moderately Agree - 7, Largely Agree - 8, and Completely Agree - 9. If the two parts cannot be separated, use the same responses for both the Inventory and AR questions, based on the combined content of the two sections."
                    "Statement 1. The auditor demonstrates high relativism by evaluating moral issues in the context of specific situations rather than following consistent moral rules."
                    "Statement 2. The auditor demonstrates high idealism by believing that desirable outcomes can always be achieved through the 'right' actions"
                    "Provide your 1 to 9 integer responses from the second task in JSON format as below:"
                    "{\"Relativism_inventory\": 0, \"Idealism_inventory\": 0, \"Relativism_AR\": 0, \"Idealism_AR\": 0}"
                )
            },
            {
                "role": "user",
                "content": text
            }
        ],
        temperature=0.7,
        max_tokens=100,
        top_p=1
    )

    json_object = json.loads(response.choices[0].message.content)
    return json_object

# List to store extracted values
data = []

# Iterate over each item in trimmed_text_list and get assessment scores
for text in text_list:
    assessment_scores = get_auditor_assessment_scores(text)
    data.append([assessment_scores["Relativism_inventory"], assessment_scores["Idealism_inventory"], assessment_scores["Relativism_AR"], assessment_scores["Idealism_AR"]])
    print(f"Relativism_inventory: {assessment_scores['Relativism_inventory']}, Idealism_inventory: {assessment_scores['Idealism_inventory']}, Relativism_AR: {assessment_scores['Relativism_AR']}, Idealism_AR: {assessment_scores['Idealism_AR']}")

# Save data to CSV file
with open('/content/drive/MyDrive/EPQ & NEG/data/OpenAI API/auditor_planning_inv_ar_epq_scores_0808v2.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Relativism_inventory', 'Idealism_inventory', 'Relativism_AR', 'Idealism_AR'])
    writer.writerows(data)

print("Data saved.")

Relativism_inventory: 3, Idealism_inventory: 7, Relativism_AR: 3, Idealism_AR: 7
Relativism_inventory: 5, Idealism_inventory: 4, Relativism_AR: 5, Idealism_AR: 4
Relativism_inventory: 7, Idealism_inventory: 4, Relativism_AR: 7, Idealism_AR: 4
Relativism_inventory: 4, Idealism_inventory: 3, Relativism_AR: 4, Idealism_AR: 3
Relativism_inventory: 4, Idealism_inventory: 7, Relativism_AR: 4, Idealism_AR: 7
Relativism_inventory: 5, Idealism_inventory: 7, Relativism_AR: 5, Idealism_AR: 7
Relativism_inventory: 5, Idealism_inventory: 7, Relativism_AR: 5, Idealism_AR: 7
Relativism_inventory: 7, Idealism_inventory: 4, Relativism_AR: 8, Idealism_AR: 5
Relativism_inventory: 5, Idealism_inventory: 6, Relativism_AR: 5, Idealism_AR: 6
Relativism_inventory: 6, Idealism_inventory: 7, Relativism_AR: 6, Idealism_AR: 7
Relativism_inventory: 4, Idealism_inventory: 8, Relativism_AR: 4, Idealism_AR: 8
Relativism_inventory: 5, Idealism_inventory: 6, Relativism_AR: 5, Idealism_AR: 6
Relativism_inventory: 5, Ide

In [ ]:
import csv
import json
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Function to get auditor assessment scores from OpenAI API
def get_auditor_assessment_scores(text):
    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an assistant helping to evaluate the performance of an auditor based on the provided audit planning documentation."
                    "Background: There are two major dimensions underlying individual variations in moral judgments. Individuals high on relativism reject the possibility of formulating or relying on universal moral rules when drawing conclusions about moral questions. Individuals high on idealism assume that desirable consequences can, with the 'right' action, always be obtained. "
                    "Your first task (do NOT print results): Separate the negotiation transcript into two parts: one for inventory write-off, the other for the reduction in allowance for doubtful accounts of accounts receivable (AR). Your second task (print results in the designated format): For each part, answer the following questions on a scale of Completely Disagree - 1, Largely Disagree - 2, Moderately Disagree - 3, Slightly Disagree - 4, Neither Agree nor Disagree - 5, Slightly Agree - 6, Moderately Agree - 7, Largely Agree - 8, and Completely Agree - 9. If the two parts cannot be separated, use the same responses for both the Inventory and AR questions, based on the combined content of the two sections."
                    "Statement 1. The auditor demonstrates high relativism by evaluating moral issues in the context of specific situations rather than following consistent moral rules."
                    "Statement 2. The auditor demonstrates high idealism by believing that desirable outcomes can always be achieved through the 'right' actions"
                    "Provide your 1 to 9 integer responses from the second task in JSON format as below:"
                    "{\"Relativism_inventory\": 0, \"Idealism_inventory\": 0, \"Relativism_AR\": 0, \"Idealism_AR\": 0}"
                )
            },
            {
                "role": "user",
                "content": text
            }
        ],
        temperature=0.7,
        max_tokens=100,
        top_p=1
    )

    json_object = json.loads(response.choices[0].message.content)
    return json_object

# List to store extracted values
data = []

# Iterate over each item in trimmed_text_list and get assessment scores
for text in text_list:
    assessment_scores = get_auditor_assessment_scores(text)
    data.append([assessment_scores["Relativism_inventory"], assessment_scores["Idealism_inventory"], assessment_scores["Relativism_AR"], assessment_scores["Idealism_AR"]])
    print(f"Relativism_inventory: {assessment_scores['Relativism_inventory']}, Idealism_inventory: {assessment_scores['Idealism_inventory']}, Relativism_AR: {assessment_scores['Relativism_AR']}, Idealism_AR: {assessment_scores['Idealism_AR']}")

# Save data to CSV file
with open('/content/drive/MyDrive/EPQ & NEG/data/OpenAI API/auditor_planning_inv_ar_epq_scores_0808v3.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Relativism_inventory', 'Idealism_inventory', 'Relativism_AR', 'Idealism_AR'])
    writer.writerows(data)

print("Data saved.")

Relativism_inventory: 3, Idealism_inventory: 4, Relativism_AR: 3, Idealism_AR: 4
Relativism_inventory: 6, Idealism_inventory: 4, Relativism_AR: 6, Idealism_AR: 4
Relativism_inventory: 3, Idealism_inventory: 4, Relativism_AR: 3, Idealism_AR: 4
Relativism_inventory: 7, Idealism_inventory: 4, Relativism_AR: 7, Idealism_AR: 4
Relativism_inventory: 1, Idealism_inventory: 8, Relativism_AR: 1, Idealism_AR: 8
Relativism_inventory: 3, Idealism_inventory: 7, Relativism_AR: 3, Idealism_AR: 7
Relativism_inventory: 6, Idealism_inventory: 7, Relativism_AR: 6, Idealism_AR: 7
Relativism_inventory: 7, Idealism_inventory: 6, Relativism_AR: 7, Idealism_AR: 6
Relativism_inventory: 7, Idealism_inventory: 4, Relativism_AR: 7, Idealism_AR: 4
Relativism_inventory: 7, Idealism_inventory: 6, Relativism_AR: 7, Idealism_AR: 6
Relativism_inventory: 3, Idealism_inventory: 7, Relativism_AR: 3, Idealism_AR: 7
Relativism_inventory: 6, Idealism_inventory: 7, Relativism_AR: 6, Idealism_AR: 7
Relativism_inventory: 6, Ide